# TextRank 알고리즘 적용 위한 스텝
1. 텍스트 수집 (완)
2. 전처리
  - 불용어 처리 (re 사용)
3. tf-idf 통해 문서 내 특정 단어의 중요도 분석
  - 텍스트를 문장 단위로 분리
  - koNLPY twitter 활용해 형태소 태깅 + 조사 및 어간 분리 통해 명사 반환
  - TfidfVectorizer()와 CountVectorizer() 활용하여 tf, df 계산
    - 이때 word2vec, doc2vec 등을 결합하여 문서 간 유사도 계산 가능한 점 참고하기

4. tf-idf 과정을 통해 얻은 문장(단어) 중요도를 활용하여 ranking, 요약할 문장(단어) 수만큼 출력

공지 파일 결측치 41개 어떻게 처리할 것인가? --> 이미지의 손상으로 인해 데이터를 보완할 수 없기 때문에 삭제하기로 결정
- 만약 HUFS 공지 검색을 통해 보완할 수 있다면 최대한 보완할 것인가? 논의하기
- 보완 +로그 남기기 + 더이상 보완할 수 없을 경우 '직접 접속해서 확인해주세요.' 출력하기

2024.04.19
- 공지 파일 결측치 중 보완 가능한 23개 내용 보충 완료 후 재업로드
- 학사 인덱스 보충완료

### 할 순서(0425 회의 이후~)
1. 전번 등 임의 전처리 해서 데이터 전처리
- copy 시 코로나 관련 공지 빼기 (문법이 파괴되지 않는 한에서 한글 표현만 최대한 남기기)
2. kospacing 적용 _ 띄어쓰기 적용
3. 형태소 분석기 (Okt())로 형태소 단위 분리
3-1. 한글자 제거?
4. 불용어 리스트 가져와서 전처리(조사 등많이 제거될 것)
5. 빈도 확인후 전처리
- 최소이하 제거 or 빈도가 지나치게 높은 counter에 대해 임의 제거
- 잘못 분리된 단어 확인하여 사용자단어에 추가
6. TF-IDF 해보기
7. 전처리 반복

- 완벽한 전처리 결론이 목요일까지 나오는 것이 목표

- 참고한 전처리
https://haystar.tistory.com/11

- hanspell 관련확인 _clone도 안됨. 해결 코드가 있긴한데 일단 후순위로 미루고 작업
https://velog.io/@woalsgodqhr/%EB%84%A4%EC%9D%B4%EB%B2%84-%EB%A7%9E%EC%B6%A4%EB%B2%95-%EA%B2%80%EC%82%AC%EA%B8%B0-api-%EC%9C%A0%ED%9A%A8%ED%95%9C-%ED%82%A4%EA%B0%80-%EC%95%84%EB%8B%99%EB%8B%88%EB%8B%A4-%EC%98%A4%EB%A5%98-%ED%95%B4%EA%B2%B0%ED%95%98%EA%B8%B0

In [ ]:
# 공지 csv의 결측치 파악
# df2[df2['content'] == '\n\n']

In [ ]:
import pandas as pd
import os

In [ ]:
#경로확인
print(os.getcwd())

/content


In [ ]:
new_dir = '/content'

# 작업 디렉토리 변경
os.chdir(new_dir)

### 마운트 및 파일 로드부터 시작하기 (경로 설정)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: 

In [ ]:
#kospacing 받기
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

In [ ]:
#작동확인
sent = '김철수는 극중 두 인격의 사나이 이광수 역을 맡았다. 철수는 한국 유일의 태권도 전승자를 가리는 결전의 날을 앞두고 10년간 함께 훈련한 사형인 유연재(김광수 분)를 찾으러 속세로 내려온 인물이다.'
new_sent = sent.replace(" ", '') # 띄어쓰기가 없는 문장 임의로 만들기
print(new_sent)

In [ ]:
from pykospacing import Spacing
spacing = Spacing()
kospacing_sent = spacing(new_sent)
#결과일치시 맞음
print(sent)
print(kospacing_sent)

In [ ]:
#결과일치시 맞음
print(sent)
print(kospacing_sent)

In [ ]:
gongji = pd.read_csv('./drive/MyDrive/csv모음/merge_gongji.csv')
haksa = pd.read_csv('./drive/MyDrive/csv모음/merged_haksa.csv')
hiring = pd.read_csv('./drive/MyDrive/csv모음/merged_hire.csv')
janghak = pd.read_csv('./drive/MyDrive/csv모음/merged_janghak.csv')
#공지가 너무 커 세개로 나눔
gongji_1 = pd.read_csv('./drive/MyDrive/csv모음/gongji_1.csv')
gongji_2 = pd.read_csv('./drive/MyDrive/csv모음/gongji_2.csv', encoding = "cp949")
gongji_3 = pd.read_csv('./drive/MyDrive/csv모음/gongji_3.csv')
gongji.head(5)

# re 이용해 특정 문자열 제거

In [ ]:
# 원본 데이터 보존 위해 copy 데이터 생성
gongji_copy = gongji.copy()
haksa_copy = haksa.copy()
hiring_copy = hiring.copy()
janghak_copy = janghak.copy()


gongji1_copy = gongji_1.copy()
gongji2_copy = gongji_2.copy()
gongji3_copy = gongji_3.copy()

In [ ]:
haksa_copy.loc[5,'content']

### 전처리과정
- 전번, 주소, 메일 등 정보 부터 지우는 중
- 가독성 때문에 띄어쓰기 및 특수기호는 문자열 지운 이후에 마지막에 지울 듯 함
- 현재 test만 지웠고 데이터 프레임에 적용하지 않았음

In [ ]:
import re

In [ ]:
gongji_copy.loc[0,'content']

In [ ]:
# 전화번호를 포함하는 패턴
tel_number = r'TEL\.\d{2}-?\d{3,4}-\d{4}'
#전번 양식 (번호만 있는 경우대비)
phone_number = r'\d{2}-\d{4}-\d{4}'
#팩스 양식
fax_number = r'FAX\.\d{2}-\d{3}-\d{4}'
#주소 양식
address = r'서울시 동대문구 이문로 107'
#이메일 양식
email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
#링크 양식
link_http = r'http://\S+'
link_www = r'\bwww\.\S*'
link_hufs = r'hufs\.ac\.kr'

#문장부호 지우기
punctuation_pattern = r'[,.:!?\\|※‘’”“→-]'

In [ ]:
test = gongji_copy.loc[0,'content']
print(test)

In [ ]:
# 각 패턴에 대해 검열 수행
test = re.sub(tel_number, '', test)
test = re.sub(phone_number, '', test)
test = re.sub(fax_number, '', test)
test = re.sub(address,'',test)
test = re.sub(email_pattern,'',test)
test = re.sub(link_http,'',test)
test = re.sub(link_www,'',test)
test = re.sub(link_hufs,'',test)
test = re.sub(punctuation_pattern,'',test)
print(test)

In [ ]:
def preprocess_text(text):
    # 링크 제거 (http:// 또는 https:// 로 시작하는 링크)
    text = re.sub(r'https?://\S+', '', text)
    # Zoom 회의 참가 ID 제거 _ 작동안함
    #text = re.sub(r'Zoom 회의 참가 ID \d{3} \d{3} \d{4}', '', text)
    # 전화번호 제거 (괄호 선택적으로)
    text = re.sub(r'\(?0\d{2,3}\)?[- ]?\d{3,4}[- ]?\d{4}', '', text)
    # 전번 제거
    text = re.sub(r'\d{2}-\d{4}-\d{4}', '', text)
    # 팩스번호 제거
    text = re.sub(r'FAX\.\d{2}-\d{3}-\d{4}', '', text)
    # 주소 제거
    text = re.sub(r'서울시 동대문구 이문로 107', '', text)
    # 이메일 제거
    text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '', text)
    # www 링크 제거
    text = re.sub(r'\bwww\.\S*', '', text)
    # hufs.ac.kr 링크 제거
    text = re.sub(r'hufs\.ac\.kr', '', text)
    # 문장부호 제거
    text = re.sub(r'[,.:!?\\|※‘’”“→-○☏『』]', '', text)

    return text


# 'content' 열에 전처리 함수를 적용하여 새로운 열 생성
gongji2_copy['cleaned_content'] = gongji2_copy['content'].apply(preprocess_text)

In [ ]:
print(gongji2_copy.loc[300, 'cleaned_content'])

In [ ]:
spacing = Spacing()

# 띄어쓰기 함수 적용
gongji2_copy['cleaned_content'] = gongji2_copy['cleaned_content'].apply(lambda x: spacing(x))

# 결과 확인
print( gongji2_copy.loc[300, 'cleaned_content'])
print( gongji2_copy.loc[300, 'cleaned_content'])


In [ ]:
# 결과 확인
print( gongji2_copy.loc[121, 'cleaned_content'])
print( gongji2_copy.loc[121, 'cleaned_content'])


### gongji2-> gongji_2_spacing.csv
- 기본 전처리 후 띄어쓰기 적용한 것 저장해둔 상태
- 전처리 좀 더 하고 품사태깅(okt)
- 빈도확인 및 사용자 사전 만들기
- TF_IDF 만들어보기

In [ ]:
# 결과를 CSV 파일로 저장
gongji2_copy.to_csv('gongji_2_spacing.csv', index=False)

# 저장된 파일 경로 확인
print("File saved asgongji_2_spacing.csv")

In [ ]:
pwd

In [ ]:
test = pd.read_csv('./drive/MyDrive/csv모음/gongji_2_spacing.csv')
test.head(10)

In [ ]:
test.loc[5,'cleaned_content']

In [ ]:
#복사
pre_gongji2_copy = test.copy()
print(pre_gongji2_copy.loc[501,'cleaned_content'])
print("####################################")
print(pre_gongji2_copy.loc[501,'content'])

### 0428~
- 정제된 텍스트 바탕으로 Okt뽑기시작

In [ ]:
#1. Konlpy에서 Okt를 불러옵니다.
!pip install konlpy
#2. konlpy 실행을 위한 자바어쩌구 다운
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#3. 필요한 설정
import os
#os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
from konlpy.tag import Okt

okt = Okt()
okt.morphs("한국외국어대학교에서 안내드립니다. 오늘은 학교수업이 없습니다.", stem = True)

In [ ]:
#복사
pre_gongji2_copy = test.copy()
print(pre_gongji2_copy.loc[501,'cleaned_content'])
print("####################################")
print(pre_gongji2_copy.loc[501,'content'])

In [ ]:
# 부동 소수점 값을 문자열로 변환하여 데이터프레임 열의 모든 값이 문자열로 구성되도록 함
pre_gongji2_copy['cleaned_content'] = pre_gongji2_copy['cleaned_content'].astype(str)

# 변환된 문자열 값을 사용하여 단어 리스트 생성
words = " ".join(pre_gongji2_copy['cleaned_content'].tolist())
words = okt.morphs(words, stem=True)
print(len(words))

In [ ]:
from collections import Counter
frequent = Counter(words).most_common()
frequent


In [ ]:
# 결과를 텍스트 파일로 저장
with open('word_frequency.txt', 'w', encoding='utf-8') as f:
    for word, count in frequent:
        f.write(f'{word}: {count}\n')
from google.colab import files
files.download('word_frequency.txt')


### 학사
- 공지 프로세스 이해를 위한 전처리 및 Tf-IDF까지 일단 가보기
- 전처리 리스트가 어느정도 완성될것이라 기대하며ㅠㅠ

In [ ]:
haksa_copy

In [ ]:
import re

### 학사 전처리 유의사항 적는 곳
- 이중전공 사전에 넣어줄 것: 이중전공을 잘 인식못함
- 이수도 넣어주기 (이/수로 인식함) _ 이수, 이수구분
- 미배정도 미/배정으로 끊김
- 학년도를 학년/도(조사)로 인식함

In [ ]:
from pykospacing import Spacing
spacing = Spacing()
kospacing_sent = spacing(haksa_copy.loc[0,'content'])
print(haksa_copy.loc[0,'content'])
print(kospacing_sent)

In [ ]:
def preprocess_text(text):
    # 링크 제거 (http:// 또는 https:// 로 시작하는 링크)
    text = re.sub(r'https?://\S+', '', text)
    # Zoom 회의 참가 ID 제거 _ 작동안함
    #text = re.sub(r'Zoom 회의 참가 ID \d{3} \d{3} \d{4}', '', text)
    # 전화번호 제거 (괄호 선택적으로)
    text = re.sub(r'\(?0\d{2,3}\)?[- ]?\d{3,4}[- ]?\d{4}', '', text)
    # 전번 제거
    text = re.sub(r'\d{2}-\d{4}-\d{4}', '', text)
    # 팩스번호 제거
    text = re.sub(r'FAX\.\d{2}-\d{3}-\d{4}', '', text)
    # 주소 제거
    text = re.sub(r'서울시 동대문구 이문로 107', '', text)
    # 이메일 제거
    text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '', text)
    # www 링크 제거
    text = re.sub(r'\bwww\.\S*', '', text)
    # hufs.ac.kr 링크 제거
    text = re.sub(r'hufs\.ac\.kr', '', text)
    #마지막 날짜 양식 변형
    #text = re.sub(r'\d{4}\.\s\d{2}\.\s\d{2}\.', '', text)
    #마지막 날짜
    #text = re.sub(r'\d{4}\.\s\d{2}\.', '', text) #과제 제출등에 유의
    # 문장부호 제거
    text = re.sub(r'[,.:!?\\|※‘’”“→-○☏『』)(]', '', text)
    #학사한정: 교무처장 제거
    text = re.sub(r'교무처장', '', text)
    text = re.sub(r'\n','',text)

    return text


# 'content' 열에 전처리 함수를 적용하여 새로운 열 생성
haksa_copy['cleaned_content'] = haksa_copy['content'].apply(preprocess_text)
haksa_copy.tail()

In [ ]:
print(haksa_copy.loc[118, 'cleaned_content'])
print("________________________________________")
print(haksa_copy.loc[118, 'content'])

In [ ]:
# kospacing 안하고 okt 넣어보기
#1. Konlpy에서 Okt를 불러옵니다.
!pip install konlpy
#2. konlpy 실행을 위한 자바어쩌구 다운
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#3. 필요한 설정
import os
#os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
from konlpy.tag import Okt

okt = Okt()
okt.morphs("한국외국어대학교에서 안내드립니다. 오늘은 학교수업이 없습니다.", stem = True)

In [ ]:
#빈도확인용
okt = Okt()
words = " ".join( haksa_copy['cleaned_content'].tolist())
words = okt.morphs(words,stem=True)

In [ ]:
#품사태그로 쪼개주기
okt = Okt()
tagged_words = []
for sentence in haksa_copy['cleaned_content']:
    tagged_sentence = okt.pos(sentence, stem=True)
    tagged_words.append(tagged_sentence)

tagged_words

In [ ]:
#품사 없이 단어만
okt = Okt()
words_list = []

for sentence in haksa_copy['cleaned_content']:
    words = okt.morphs(sentence, stem=True)
    words_list.append(words)


In [ ]:
#태그된 단어 일단 메모장에넣어주기
from google.colab import files

with open('haksa_tagged_words.txt', 'w', encoding='utf-8') as file:
    for sentence in tagged_words:
        line = ' '.join([f'{word}/{tag}' for word, tag in sentence])
        file.write(line + '\n')

files.download('haksa_tagged_words.txt')


In [ ]:
#품사없이 단어만 일단 메모장에넣어주기
from google.colab import files

# 파일을 쓰기 모드로 열고 데이터를 쓴 후 파일을 다운로드합니다.
with open('haksa_words_list.txt', 'w', encoding='utf-8') as file:
    for words in words_list:
        line = ' '.join(words)
        file.write(line + '\n')

# 다운로드할 파일을 지정합니다.
files.download('haksa_words_list.txt')

In [ ]:
words[4]

In [ ]:
from collections import Counter
frequent = Counter(words).most_common()
frequent

### tf-idf 참고링크
https://heytech.tistory.com/337


In [ ]:
#tf-idf 넣어줄 사이킷런다운
!pip install -U scikit-learn

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
#판다스로열어줌
documents = pd.read_csv("haksa_words_list.txt")
documents

In [ ]:
#공지 읽어오기
with open("haksa_words_list.txt",'r',encoding = 'utf-8') as file:
  lines = file.readlines()
  data = [line.strip() for line in lines]
documents = data
documents[1]

In [ ]:
vectorizer = CountVectorizer()
# Document Term Matrix
dtm = vectorizer.fit_transform(documents)

# Term Freqeuncy
tf = pd.DataFrame(dtm.toarray(), columns=vectorizer.get_feature_names_out())
# Document Frequency
df = tf.astype(bool).sum(axis = 0)
# 문서 개수
D = len(tf)

# Inverse Document Frequency
idf = np.log((D) / (df+1))
# TF-IDF
tfidf = tf * idf
tfidf = tfidf / np.linalg.norm(tfidf, axis = 1, keepdims = True)
tfidf

In [ ]:
tf

In [ ]:
df

In [ ]:
D

In [ ]:
tfidf

# 코드 정리지점
### 다시 공지 전처리 해보고 tf-idf 해보기

In [3]:
#호출코드모음
import pandas as pd
import os
#마운트
from google.colab import drive
drive.mount('/content/drive')
import re
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
#단어 빈도셀 때
from collections import Counter
import string

Mounted at /content/drive


In [ ]:
#kospacing 다운 및 호출
#!pip install git+https://github.com/haven-jeon/PyKoSpacing.git
#from pykospacing import Spacing
#spacing = Spacing()
#kospacing_sent = spacing(new_sent)

In [5]:
#konlpy 다운 및 호출
#1. Konlpy에서 Okt
!pip install konlpy
#2. konlpy 실행을 위한 자바어쩌구 다운
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#3. 필요한 설정
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
from konlpy.tag import Okt

okt = Okt()
okt.morphs("한국외국어대학교에서 안내드립니다. 오늘은 학교수업이 없습니다.", stem = True)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 41.0 MB/s eta 0:00:00


['한국',
 '외국어',
 '대학교',
 '에서',
 '안',
 '내',
 '드리다',
 '.',
 '오늘',
 '은',
 '학교',
 '수업',
 '이',
 '없다',
 '.']

In [ ]:
okt.morphs("안내 합니다")
okt.morphs("안내 드립니다")
okt.morphs("안내드립니다")

['안', '내', '드립니다']

In [ ]:
#각종 문서 저장

gongji = pd.read_csv('./drive/MyDrive/csv모음/final_merged_gongji(0501).csv')
haksa = pd.read_csv('./drive/MyDrive/csv모음/merged_haksa.csv')
hiring = pd.read_csv('./drive/MyDrive/csv모음/merged_hire.csv')
janghak = pd.read_csv('./drive/MyDrive/csv모음/merged_janghak.csv')

#공지 분할 저장
gongji_1 = pd.read_csv('./drive/MyDrive/csv모음/final_gongji_1.csv')
gongji_2 = pd.read_csv('./drive/MyDrive/csv모음/final_gongji_2.csv')
gongji_3 = pd.read_csv('./drive/MyDrive/csv모음/final_gongji_3.csv')
gongji_4 = pd.read_csv('./drive/MyDrive/csv모음/final_gongji_4.csv')

In [ ]:
#공지 4분할

# DataFrame 분할
'''splitted_dfs = np.array_split(gongji, 4)

# 각각의 DataFrame을 따로 저장할 수도 있습니다
for i, splitted_df in enumerate(splitted_dfs):
    splitted_df.to_csv(f'final_gongji_{i+1}.csv', index=False)'''

"splitted_dfs = np.array_split(gongji, 4)\n\n# 각각의 DataFrame을 따로 저장할 수도 있습니다\nfor i, splitted_df in enumerate(splitted_dfs):\n    splitted_df.to_csv(f'final_gongji_{i+1}.csv', index=False)"

In [ ]:
#각 공지 1677개씩 있음
len(gongji_4)

1677

In [ ]:
#파일복사
# 원본 데이터 보존 위해 copy 데이터 생성
gongji_copy = gongji.copy()
haksa_copy = haksa.copy()
hiring_copy = hiring.copy()
janghak_copy = janghak.copy()


gongji1_copy = gongji_1.copy()
gongji2_copy = gongji_2.copy()
gongji3_copy = gongji_3.copy()
gongji4_copy = gongji_4.copy()

In [ ]:
#전처리 함수(통일)
def preprocess_text(text):
    # 링크 제거 (http:// 또는 https:// 로 시작하는 링크)
    text = re.sub(r'https?://\S+', '', text)
    # Zoom 회의 참가 ID 제거 _ 작동안함
    #text = re.sub(r'Zoom 회의 참가 ID \d{3} \d{3} \d{4}', '', text)
    # 전화번호 제거 (괄호 선택적으로)
    text = re.sub(r'\(?0\d{2,3}\)?[- ]?\d{3,4}[- ]?\d{4}', '', text)
    # 전번 제거
    text = re.sub(r'\d{2}-\d{4}-\d{4}', '', text)
    # 팩스번호 제거
    text = re.sub(r'FAX\.\d{2}-\d{3}-\d{4}', '', text)
    # 주소 제거
    text = re.sub(r'서울시 동대문구 이문로 107', '', text)
    # 이메일 제거
    text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '', text)
    # www 링크 제거
    text = re.sub(r'\bwww\.\S*', '', text)
    # hufs.ac.kr 링크 제거
    text = re.sub(r'hufs\.ac\.kr', '', text)
    # 문장부호 제거
    text = re.sub(r'[,.:!?\\|※‘’”“→-○☏『』]', '', text)

    return text

# 학사처리
1. 기본 양식 re로 전처리
2. 코로나 게시글 제거 (제거한 데이터프레임 따로저장?)
3. 사용자 사전 추가
4. 빈도수 확인 후 사용자 사전 추가
5. 한글자 지우기
6. 품사태깅해서 조사, 동사 등등 지우기
7. TF-IDF 연산
8. 다시 사용자 사전 추가


In [ ]:
haksa = pd.read_csv('./drive/MyDrive/csv모음/merged_haksa.csv')
haksa_copy = haksa.copy()
haksa_copy

,index,date,title,content
0,1,2020-01-06,[공통] 2019-2학기 이중전공 변경 배정 확정 공고,\n\n\n\n\n\n[공통]이중전공변경 배정 확정 공고\n \n2019학년도 제2...
1,2,2020-01-13,[공통] 2020학년도 1학기 수강신청 공고,\n2020학년도 1학기 수강신청 공고 2020학년도 제1학기 수강신청에 대하여 아...
2,3,2020-01-13,[공통] 2020학년도 전기 국제지역대학원 학석사연계과정 모집 안내,\n2020학년도 전기 국제지역대학원 학석사연계과정 모집 안내\n \n2020학년도...
3,4,2020-01-13,[공통] 2020학년도 전기 일반대학원 학석사연계과정 모집 안내,\n2020학년도 전기 일반대학원 학석사연계과정 모집 안내\n\n2020학년도 전기...
4,5,2020-01-15,[글로벌]졸업 예비사정 결과 조회,\n[글로벌] 졸업 예비사정 결과 조회글로벌캠퍼스 졸업(2020년 2월) 예비사정 ...
...,...,...,...,...
617,618,2024-03-13,[공통] 생리 유고 신청 안내,\n\n\n\n\n\n생리 유고 신청 안내\n\n\n\n\n\n 생리로 인한 유고...
618,619,2024-03-13,[공통] 졸업예정자(해당학기 졸업가능자)의 조기취업 유고결석 신청 안내,\n\n\n\n\n\n졸업예정자(해당학기 졸업가능자)의 조기취업유고결석 신청 안내\...
619,621,2024-03-15,[글로벌]2024-1학기 폐강(2차) 공고(글로벌캠퍼스),\n\n\n\n\n\n2024-1학기 폐강(2차) 공고(글로벌캠퍼스)\n\n\n\n...
620,620,2024-03-15,[서울] 2024-1학기 폐강(2차) 공고(서울캠퍼스),\n\n\n\n\n\n2024-1학기 폐강(2차) 공고(서울캠퍼스)\n\n\n\n\...


In [ ]:
def preprocess_haksa(text):
    # 링크 제거 (http:// 또는 https:// 로 시작하는 링크)
    text = re.sub(r'https?://\S+', '', text)
    # Zoom 회의 참가 ID 제거 _ 작동안함
    #text = re.sub(r'Zoom 회의 참가 ID \d{3} \d{3} \d{4}', '', text)
    # 전화번호 제거 (괄호 선택적으로)
    text = re.sub(r'\(?0\d{2,3}\)?[- ]?\d{3,4}[- ]?\d{4}', '', text)
    # 전번 제거
    text = re.sub(r'\d{2}-\d{4}-\d{4}', '', text)
    # 팩스번호 제거
    text = re.sub(r'FAX\.\d{2}-\d{3}-\d{4}', '', text)
    # 주소 제거
    text = re.sub(r'서울시 동대문구 이문로 107', '', text)
    # 이메일 제거
    text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '', text)
    # www 링크 제거
    text = re.sub(r'\bwww\.\S*', '', text)
    # hufs.ac.kr 링크 제거
    text = re.sub(r'hufs\.ac\.kr', '', text)
    #마지막 날짜 양식 변형
    #text = re.sub(r'\d{4}\.\s\d{2}\.\s\d{2}\.', '', text)
    #마지막 날짜
    #text = re.sub(r'\d{4}\.\s\d{2}\.', '', text) #과제 제출등에 유의
    # 문장부호 제거
    text = re.sub(r'[,.:!?\\|※‘’”“→-○☏『』)(]', '', text)
    text= re.sub(r'\n','',text)
    return text

In [ ]:
# 'content' 열에 전처리 함수를 적용하여 새로운 열 생성
haksa_copy['cleaned_content'] = haksa_copy['content'].apply(preprocess_haksa)
haksa_copy.loc[6,'cleaned_content']

'공    고 2020학년도 제1학기 휴·복학 신청 일정 및 방법 등을 공고합니다1 일정  2020 2 3월 ~ 2 7금2 방법  [종합정보시스템] 로그인  [휴/복학 신청]3 유의사항 가 일반 휴학 - 휴학은 1년 단위로 신청 단 1개 학기 이후 복학 가능 - 휴학기간 만료 후 다시 휴학신청 가능하며 재학 중 3년을 초과할 수 없음 단 2012년 2학기 이후 3학년으로 편입학한 학생은 2년을 초과할 수 없음  휴학기간 만료 후 복학하지 않을 경우 휴학기간초과제적됨 - 신·편입생 재입학생의 첫 학기 휴학 불가 단 질병으로 인해 4주 이상 치료를 요하는 경우에 한해서 일반휴학을 허가할 수 있음 별도문의 - 수업일수 1/4선 이후 아래의 경우에 한해서만 휴학을 허가하고 등록금 차액은 환불함  질병 또는 입원으로 인하여 4주 이상의 치료를 요하는 경우  3/4선까지만 허용 종합병원에서 발급받은 4주 이상 입원진단서 또는 4주 이상 수업참여가 어렵다는 의사소견진단서 제출  학업을 계속할 수 없는 명백한 사유가 있는 경우  중간고사 전 주 금요일까지만 허용 공식적인 서류로 사유를 증명할 수 있어야 함  증빙서류 미비 또는 요건 미충족 시 허가하지 않음 - 일반 휴학 중 입대할 경우 군입대 휴학으로 재신청하여야 함미처리 시 일반 휴학기간 종료 후 제적처리 됨 - 등록 휴학  휴학신청기간에 휴학 신청 후 등록기간에 등록금 납부 시 자동 처리됨 나 군입대 휴학 - 입대일 이전 학교 종합정보시스템에서 신청 후 입영통지서 사본 제출 이메일  군입대 휴학자 중 장기복무자는 입영통지서 제출 시 또는 장기확정 시 필히 구두로 신고하여야 함 - 휴학기간  입영일부터 전역일까지 귀향조치 시 관련 증빙서류 제출 후 조치 요망 - 전역일 이후 계속 휴학을 원하는 경우 복학예정일이 남아있더라도 일반 휴학으로 전환하여야 함 전역일이 기재되어있는 증빙서류 제출 - 1/4선 후~3/4선 이전 입대 예정자는 일반 휴학 신청 후 군입대 휴학으로 재신청 하여야 함 - 3/4선 후 입대 예정자가 금학기 

In [ ]:
word_haksa = []

for i in range(len(haksa_copy)):
    okt_pos = okt.pos(haksa_copy['cleaned_content'][i], norm=True, stem=True)

    word_list = [word[0] for word in okt_pos if word[1] not in ['Josa', 'Eomi', 'Punctuation', 'Suffix']]
    word_haksa.extend(word_list)

In [ ]:
word_haksa

['공통',
 '이중전공',
 '변경',
 '배정',
 '확정',
 '공고',
 '2019',
 '학년도',
 '제',
 '2',
 '학기',
 '이중전공',
 '변경',
 '배정',
 '결과',
 '아래',
 '같이',
 '공고함',
 '아',
 '1',
 '결과',
 '조회',
 '종합정보시스템',
 '전공',
 '교직',
 '관리',
 '이중전공',
 '변경',
 '신청',
 '안내',
 '사항',
 '2',
 '이중전공',
 '변경',
 '시',
 '배정',
 '되다',
 '학생',
 '추후',
 '이중전공',
 '변경',
 '신청',
 '불가',
 '미배정',
 '되다',
 '학생',
 '기존',
 '이',
 '수중',
 '이중전공',
 '부전',
 '공이',
 '유지',
 '되다',
 '추후',
 '이중전공',
 '변경',
 '기간',
 '중',
 '신청',
 '가능',
 '3',
 '2020년',
 '2월',
 '졸업',
 '예정자',
 '4',
 '학년',
 '수료',
 '졸업',
 '대기',
 '포함',
 '늘다',
 '반드시',
 '1월',
 '초까지',
 '이',
 '수',
 '구분',
 '변경',
 '신청',
 '하다',
 '영역',
 '별',
 '졸업',
 '학점',
 '누락',
 '되다',
 '일이',
 '없다',
 '하다',
 '함',
 '4',
 '이중전공',
 '변경',
 '따르다',
 '성적',
 '조회',
 '후',
 '변경',
 '되다',
 '않다',
 '이',
 '수',
 '구분',
 '수정',
 '학사종합지원센터',
 '내',
 '1',
 '전공',
 '소속',
 '단과대학',
 '창구',
 '방문',
 '하다',
 '이',
 '수',
 '구분',
 '변경',
 '신청',
 '작성',
 '하다',
 '제출',
 '하다',
 '함',
 '방학',
 '및',
 '학기',
 '중',
 '가능',
 '5',
 '기타',
 '사항',
 '1',
 '부',
 '전공',
 '전공',
 '심화',
 '부',
 '전공',
 '단',
 '

In [ ]:
#어떻게조개지는지확인
okt = Okt()
words = haksa_copy.loc[35,'cleaned_content']
words = okt.morphs(words,stem=True)
print(words, end = "")

['2020', '학년도', '제', '1', '학기', '폐강', '2', '차', '공고', '서울캠퍼스', '2020', '학년도', '제', '1', '학기', '서울캠퍼스', '폐강', '강좌', '에', '아래', '와', '같이', '2', '차', '공고', '하다', '폐강', '강좌', '를', '수강신청', '한', '학생', '은', '아래', '와', '같이', '해당', '기한', '내', '에', '수강신청', '을', '변경', '하다', '바라다', '-', '아', '래', '-', '폐강', '강좌', '수강생', '수강신청', '변경', '1', '2020', '학년도', '1', '학기', '폐강', '안내', '동영상', '변경', '기간', '2020', '3', '25', '수', '~', '3', '27', '금', '오후', '300', '까지', '3', '27', '금', '오후', '3시', '까지', '수신', '되다', '이메일', '에', '하다', '접다', '기한', '내', '이메일', '발송', '요망', '3', '작성', '양식', '이', '공지', '의', '첨부', '파일', '폐강', '과목', '대체', '수강', '신청서', '작성', '4', '제', '추다', '처', '본인', '1', '전공', '소속', '단과대학', '교학', '파트', '학사종합지원센터', '이메일', '제출', '소속', '단과대학', '이메일', '문의', '02-2173', '-', 'XXXX', '창구', '서양어대학', 'hana@21215', '번', '사회과학', '대학', '상경', '대학', '경영', '대학', '국제학부', 'LD', '학부', 'LT', '학부', 'KFL', '학부', 'jjlove@23086', '번영', '어', '대학', '아시아언어문화대학', '중국학', '대학', '일본학', '대학', 'paradiso21c@22647', '번', '사범대학', 'pois

In [ ]:
okt = Okt()
frequent = Counter(word_haksa).most_common()
frequent

[('하다', 5271),
 ('전공', 3868),
 ('1', 3667),
 ('신청', 3648),
 ('2', 3400),
 ('학과', 2844),
 ('학기', 2505),
 ('수', 2456),
 ('및', 2269),
 ('되다', 2056),
 ('3', 1943),
 ('학점', 1789),
 ('학생', 1769),
 ('대학', 1728),
 ('수강신청', 1642),
 ('4', 1530),
 ('이중전공', 1515),
 ('경우', 1478),
 ('함', 1457),
 ('수강', 1424),
 ('변경', 1403),
 ('기간', 1322),
 ('졸업', 1238),
 ('이수', 1196),
 ('6', 1150),
 ('후', 1149),
 ('가능', 1112),
 ('제출', 1098),
 ('불가', 1081),
 ('확인', 1064),
 ('금', 1014),
 ('과목', 992),
 ('납부', 983),
 ('부', 979),
 ('시', 972),
 ('5', 965),
 ('수업', 963),
 ('통번역', 951),
 ('7', 904),
 ('없다', 891),
 ('중', 875),
 ('성적', 874),
 ('해당', 846),
 ('교과목', 825),
 ('배정', 818),
 ('이', 810),
 ('제', 804),
 ('있다', 801),
 ('학부', 791),
 ('학교', 783),
 ('학년도', 773),
 ('글로벌캠퍼스', 765),
 ('취소', 745),
 ('8', 744),
 ('글로벌', 704),
 ('목', 703),
 ('종합정보시스템', 699),
 ('일정', 694),
 ('학년', 680),
 ('또는', 676),
 ('서울캠퍼스', 676),
 ('학번', 676),
 ('안내', 675),
 ('사항', 669),
 ('서울', 640),
 ('등록', 638),
 ('교양', 634),
 ('내', 631),
 ('등', 628),
 ('2

In [ ]:
'''#빈도확인용
okt = Okt()
words = " ".join( haksa_copy['cleaned_content'].tolist())
words = okt.morphs(words,stem=True)

frequent = Counter(words).most_common()
frequent'''

'#빈도확인용\nokt = Okt()\nwords = " ".join( haksa_copy[\'cleaned_content\'].tolist())\nwords = okt.morphs(words,stem=True)\n\nfrequent = Counter(words).most_common()\nfrequent'

In [ ]:
with open('./haksa_stopwords.txt', 'r') as f:
    list_file = f.readlines()
stopwords = list_file[0].split(",")
remove_stopwords = [x for x in word_haksa if x not in stopwords]
len(remove_stopwords)

224722

In [ ]:
#한글자 제거
remove_one_word = [x for x in remove_stopwords if len(x)>1]
len(remove_one_word)

185381

In [ ]:
len(word_haksa)

251363

- 전공관련 키워드 구체화
- 학기 키워드 세부화
- 학점 키워드 세부화
- 수강 변경기간 등 키워드화
- 1학기 2학기가 적용이안되나?
- 이수가능

In [ ]:
Counter(remove_one_word).most_common()

[('전공', 3868),
 ('신청', 3648),
 ('학과', 2844),
 ('학기', 2505),
 ('학점', 1789),
 ('학생', 1769),
 ('대학', 1728),
 ('수강신청', 1642),
 ('이중전공', 1515),
 ('경우', 1478),
 ('수강', 1424),
 ('변경', 1403),
 ('기간', 1322),
 ('졸업', 1238),
 ('이수', 1196),
 ('가능', 1112),
 ('제출', 1098),
 ('불가', 1081),
 ('확인', 1064),
 ('과목', 992),
 ('납부', 983),
 ('수업', 963),
 ('통번역', 951),
 ('성적', 874),
 ('해당', 846),
 ('교과목', 825),
 ('배정', 818),
 ('학부', 791),
 ('학교', 783),
 ('학년도', 773),
 ('글로벌캠퍼스', 765),
 ('취소', 745),
 ('글로벌', 704),
 ('종합정보시스템', 699),
 ('일정', 694),
 ('학년', 680),
 ('또는', 676),
 ('서울캠퍼스', 676),
 ('학번', 676),
 ('안내', 675),
 ('사항', 669),
 ('서울', 640),
 ('등록', 638),
 ('교양', 634),
 ('2021', 628),
 ('1000', 619),
 ('아래', 617),
 ('학사종합지원센터', 612),
 ('인정', 594),
 ('휴학', 593),
 ('등록금', 582),
 ('홈페이지', 567),
 ('공고', 566),
 ('2020', 558),
 ('이후', 558),
 ('2022', 549),
 ('시험', 523),
 ('재수강', 519),
 ('계절학기', 517),
 ('방법', 516),
 ('영어', 513),
 ('세부', 510),
 ('2023', 502),
 ('기준', 501),
 ('캠퍼스', 500),
 ('문의', 492),
 ('the', 488),

######################공통 구간#######################################

이 아래로 공지 병합 코딩진행

In [ ]:
gongji2_copy['title']

In [ ]:
#코로나 개수확인
corona_count = len(gongji2_copy[gongji2_copy['title'].str.contains('코로나19')])
# 결과 출력
print("코로나19가 포함된 데이터 개수:", corona_count)
print("삭제 전 데이터 개수:", len(gongji2_copy))

In [ ]:
#공지2 코로나 행 삭제
drop_covid_2 = gongji2_copy[gongji2_copy['title'].str.contains('코로나19')]
gongji2_copy.drop(drop_covid_2.index, inplace=True)
print("삭제 후 데이터 개수:", len(gongji2_copy))

In [ ]:
#공지1 코로나 행 삭제
#코로나 개수확인
corona_count = len(gongji1_copy[gongji1_copy['title'].str.contains('코로나19')])
# 결과 출력
print("코로나19가 포함된 데이터 개수:", corona_count)
print("삭제 전 데이터 개수:", len(gongji1_copy))

drop_covid_1 = gongji1_copy[gongji1_copy['title'].str.contains('코로나19')]
gongji1_copy.drop(drop_covid_1.index, inplace=True)
print("삭제 후 데이터 개수:", len(gongji1_copy))

In [ ]:
#'content' 열에 전처리 함수를 적용하여 새로운 열 생성
gongji2_copy['cleaned_content'] = gongji2_copy['content'].apply(preprocess_text)
gongji2_copy.tail()

In [ ]:
#빈도확인용
okt = Okt()
words = " ".join( gongji2_copy['cleaned_content'].tolist())
words = okt.morphs(words,stem=True)
words

In [ ]:
frequent = Counter(words).most_common()
frequent

In [ ]:
pwd

In [ ]:
#코로나 개수확인
corona_count = len(haksa_copy[haksa_copy['title'].str.contains('코로나19')])
# 결과 출력
print("코로나19가 포함된 데이터 개수:", corona_count)
print("삭제 전 데이터 개수:", len(haksa_copy))

# 공지처리

## 공지1

In [ ]:
gongji1_copy

,index,date,title,content
0,4597,2024.03.27,[글로벌] 2024-1 명상 집단상담 ＇금모닝 마음챙김＇ 프로그램 안내,\n[글로벌] 2024-1 명상 집단상담 ＇금모닝 마음챙김＇ 프로그램 안내\n\n학...
1,4596,2024.03.27,[서울] 보건실 - 심폐소생술 교육일정 안내 (2024.04.29(월) 10시 / ...,\n심폐소생술 교육일정 안내\n\n서울캠퍼스 보건실에서는 교내 구성원의 응급처치 능...
2,4595,2024.03.26,"EU연구소 인문사회연구사업단, '제5회 바우하우스 콜로키움' 개최","\nEU연구소 인문사회연구사업단, '제5회 바우하우스 콜로키움' 개최\n\n\nEU..."
3,4594,2024.03.26,인도연구소 HK+사업단 제66회 명사초청 특강 안내,\n인도연구소 HK+사업단 제66회 명사초청 특강 안내\n\n□한국외국어대학교 인도...
4,4593,2024.03.25,FBS 외대교육방송국 78기 수습국원 추가모집,\nFBS 외대교육방송국 78기 수습국원 추가모집\n\n한국외국어대학교FBS외대교육...
...,...,...,...,...
1672,2925,2021.10.01,[서울] 교직원을 위한 학생 상담 특강 안내,\n안녕하세요 (서울) 학생상담센터입니다.학생상담센터에서 '교직원을 위한 학생 상담...
1673,2924,2021.10.01,코로나19 발병현황 및 행동지침 (2021.10.01),\n코로나19 발병현황 및 행동지침 (2021.10.01) 1. 일일 확진자 현황 ...
1674,2923,2021.10.01,코로나19 관련 현황 및 조치사항(2021.10.01),\n코로나19 관련 현황 및 조치사항(2021.10.01)1. 부서별 주요 조치사항...
1675,2922,2021.10.01,2021학년도 2학기 교직원버스 운행표 변경,\n2021학년도 2학기 교직원버스 운행표\n(코로나 4단계 사회적 거리두기 연장에...


In [ ]:
#코로나 제거 전 개수확인
corona_count = len(gongji1_copy[gongji1_copy['title'].str.contains('코로나19')])
# 결과 출력
print("코로나19가 포함된 데이터 개수:", corona_count)
print("삭제 전 데이터 개수:", len(gongji1_copy))

코로나19가 포함된 데이터 개수: 350
삭제 전 데이터 개수: 1677


In [ ]:
#코로나 제거 (~)
gongji1_copy = gongji1_copy[~gongji1_copy['title'].str.contains('코로나19')]
# 1327 = 1677 - 350
print("코로나 19 삭제된 gongji1 개수",len(gongji1_copy))

코로나 19 삭제된 gongji1 개수 1327


In [ ]:
gongji1_copy

,index,date,title,content
0,4597,2024.03.27,[글로벌] 2024-1 명상 집단상담 ＇금모닝 마음챙김＇ 프로그램 안내,\n[글로벌] 2024-1 명상 집단상담 ＇금모닝 마음챙김＇ 프로그램 안내\n\n학...
1,4596,2024.03.27,[서울] 보건실 - 심폐소생술 교육일정 안내 (2024.04.29(월) 10시 / ...,\n심폐소생술 교육일정 안내\n\n서울캠퍼스 보건실에서는 교내 구성원의 응급처치 능...
2,4595,2024.03.26,"EU연구소 인문사회연구사업단, '제5회 바우하우스 콜로키움' 개최","\nEU연구소 인문사회연구사업단, '제5회 바우하우스 콜로키움' 개최\n\n\nEU..."
3,4594,2024.03.26,인도연구소 HK+사업단 제66회 명사초청 특강 안내,\n인도연구소 HK+사업단 제66회 명사초청 특강 안내\n\n□한국외국어대학교 인도...
4,4593,2024.03.25,FBS 외대교육방송국 78기 수습국원 추가모집,\nFBS 외대교육방송국 78기 수습국원 추가모집\n\n한국외국어대학교FBS외대교육...
...,...,...,...,...
1668,2929,2021.10.05,[서울] 중국인 유학생의 심리 및 학업 적응을 위한 워크샵 안내 - 中国留学生的心理...,\n안녕하세요 (서울)학생상담센터입니다.(서울)학생상담센터에서 외대에 재학중인 중국...
1670,2927,2021.10.05,(21-2)한국어존(Korean Zone)참여 안내,\n2021학년도2학기 한국어존(K-Zone)을 다음과 같이 운영합니다.글로벌캠퍼스...
1671,2926,2021.10.01,[서울] 중국인 유학생의 심리 및 학업 적응을 위한 워크샵 안내 - 中国留学生的心理...,\n안녕하세요 (서울)학생상담센터입니다.(서울)학생상담센터에서 외대에 재학중인 중국...
1672,2925,2021.10.01,[서울] 교직원을 위한 학생 상담 특강 안내,\n안녕하세요 (서울) 학생상담센터입니다.학생상담센터에서 '교직원을 위한 학생 상담...


In [ ]:
def preprocess_gongji(text):
    # 링크 제거 (http:// 또는 https:// 로 시작하는 링크)
    text = re.sub(r'https?://\S+', '', text)
    # Zoom 회의 참가 ID 제거 _ 작동안함
    #text = re.sub(r'Zoom 회의 참가 ID \d{3} \d{3} \d{4}', '', text)
    # 전화번호 제거 (괄호 선택적으로)
    text = re.sub(r'\(?0\d{2,3}\)?[- ]?\d{3,4}[- ]?\d{4}', '', text)
    # 전번 제거
    text = re.sub(r'\d{2}-\d{4}-\d{4}', '', text)
    # 팩스번호 제거
    text = re.sub(r'FAX\.\d{2}-\d{3}-\d{4}', '', text)
    # 주소 제거
    text = re.sub(r'서울시 동대문구 이문로 107', '', text)
    # 이메일 제거
    text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '', text)
    # www 링크 제거
    text = re.sub(r'\bwww\.\S*', '', text)
    # hufs.ac.kr 링크 제거
    text = re.sub(r'hufs\.ac\.kr', '', text)
    #마지막 날짜 양식 변형
    text = re.sub(r'\d{4}\.\s\d{2}\.\s\d{2}\.', '', text)
    #마지막 날짜
    text = re.sub(r'\d{4}\.\s\d{2}\.', '', text) #과제 제출등에 유의
    # 문장부호 제거
    text = re.sub(r'[,.:!?\\|※‘’”“→-○☏『』)(]', '', text)
    text= re.sub(r'\n','',text)
    return text

# 'content' 열에 전처리 함수를 적용하여 새로운 열 생성
gongji1_copy['cleaned_content'] = gongji1_copy['content'].apply(preprocess_gongji)
gongji1_copy.tail()

<ipython-input-11-fd95e59bed66>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gongji1_copy['cleaned_content'] = gongji1_copy['content'].apply(preprocess_gongji)


,index,date,title,content,cleaned_content
1668,2929,2021.10.05,[서울] 중국인 유학생의 심리 및 학업 적응을 위한 워크샵 안내 - 中国留学生的心理...,\n안녕하세요 (서울)학생상담센터입니다.(서울)학생상담센터에서 외대에 재학중인 중국...,안녕하세요 서울학생상담센터입니다서울학생상담센터에서 외대에 재학중인 중국인 유학생을 ...
1670,2927,2021.10.05,(21-2)한국어존(Korean Zone)참여 안내,\n2021학년도2학기 한국어존(K-Zone)을 다음과 같이 운영합니다.글로벌캠퍼스...,2021학년도2학기 한국어존K-Zone을 다음과 같이 운영합니다글로벌캠퍼스 외국인 ...
1671,2926,2021.10.01,[서울] 중국인 유학생의 심리 및 학업 적응을 위한 워크샵 안내 - 中国留学生的心理...,\n안녕하세요 (서울)학생상담센터입니다.(서울)학생상담센터에서 외대에 재학중인 중국...,안녕하세요 서울학생상담센터입니다서울학생상담센터에서 외대에 재학중인 중국인 유학생을 ...
1672,2925,2021.10.01,[서울] 교직원을 위한 학생 상담 특강 안내,\n안녕하세요 (서울) 학생상담센터입니다.학생상담센터에서 '교직원을 위한 학생 상담...,안녕하세요 서울 학생상담센터입니다학생상담센터에서 '교직원을 위한 학생 상담 어떻게 ...
1675,2922,2021.10.01,2021학년도 2학기 교직원버스 운행표 변경,\n2021학년도 2학기 교직원버스 운행표\n(코로나 4단계 사회적 거리두기 연장에...,2021학년도 2학기 교직원버스 운행표코로나 4단계 사회적 거리두기 연장에 따른 임...


In [ ]:
gongji1_copy

,index,date,title,content,cleaned_content
0,4597,2024.03.27,[글로벌] 2024-1 명상 집단상담 ＇금모닝 마음챙김＇ 프로그램 안내,\n[글로벌] 2024-1 명상 집단상담 ＇금모닝 마음챙김＇ 프로그램 안내\n\n학...,[글로벌] 2024-1 명상 집단상담 ＇금모닝 마음챙김＇ 프로그램 안내학생상담센터에...
1,4596,2024.03.27,[서울] 보건실 - 심폐소생술 교육일정 안내 (2024.04.29(월) 10시 / ...,\n심폐소생술 교육일정 안내\n\n서울캠퍼스 보건실에서는 교내 구성원의 응급처치 능...,심폐소생술 교육일정 안내서울캠퍼스 보건실에서는 교내 구성원의 응급처치 능력을 향상시...
2,4595,2024.03.26,"EU연구소 인문사회연구사업단, '제5회 바우하우스 콜로키움' 개최","\nEU연구소 인문사회연구사업단, '제5회 바우하우스 콜로키움' 개최\n\n\nEU...",EU연구소 인문사회연구사업단 '제5회 바우하우스 콜로키움' 개최EU연구소 인문사회연...
3,4594,2024.03.26,인도연구소 HK+사업단 제66회 명사초청 특강 안내,\n인도연구소 HK+사업단 제66회 명사초청 특강 안내\n\n□한국외국어대학교 인도...,인도연구소 HK+사업단 제66회 명사초청 특강 안내한국외국어대학교 인도연구소소장 김...
4,4593,2024.03.25,FBS 외대교육방송국 78기 수습국원 추가모집,\nFBS 외대교육방송국 78기 수습국원 추가모집\n\n한국외국어대학교FBS외대교육...,FBS 외대교육방송국 78기 수습국원 추가모집한국외국어대학교FBS외대교육방송국은19...
...,...,...,...,...,...
1668,2929,2021.10.05,[서울] 중국인 유학생의 심리 및 학업 적응을 위한 워크샵 안내 - 中国留学生的心理...,\n안녕하세요 (서울)학생상담센터입니다.(서울)학생상담센터에서 외대에 재학중인 중국...,안녕하세요 서울학생상담센터입니다서울학생상담센터에서 외대에 재학중인 중국인 유학생을 ...
1670,2927,2021.10.05,(21-2)한국어존(Korean Zone)참여 안내,\n2021학년도2학기 한국어존(K-Zone)을 다음과 같이 운영합니다.글로벌캠퍼스...,2021학년도2학기 한국어존K-Zone을 다음과 같이 운영합니다글로벌캠퍼스 외국인 ...
1671,2926,2021.10.01,[서울] 중국인 유학생의 심리 및 학업 적응을 위한 워크샵 안내 - 中国留学生的心理...,\n안녕하세요 (서울)학생상담센터입니다.(서울)학생상담센터에서 외대에 재학중인 중국...,안녕하세요 서울학생상담센터입니다서울학생상담센터에서 외대에 재학중인 중국인 유학생을 ...
1672,2925,2021.10.01,[서울] 교직원을 위한 학생 상담 특강 안내,\n안녕하세요 (서울) 학생상담센터입니다.학생상담센터에서 '교직원을 위한 학생 상담...,안녕하세요 서울 학생상담센터입니다학생상담센터에서 '교직원을 위한 학생 상담 어떻게 ...


In [ ]:
'''#토큰화: 사용자 사전 추가시 사용
def tokenize_sentences(sentence):
    tokens = okt.pos(sentence)  # 형태소 분석 및 품사 태깅
    return tokens

# 'cleaned_content' 열에 있는 각 문장에 토큰화 및 품사 태깅 적용
gongji1_copy['tagged_content'] = gongji1_copy['cleaned_content'].apply(tokenize_sentences)

# 결과 확인
print(gongji1_copy['tagged_content'])'''

0       [([, Punctuation), (글로벌, Noun), (], Punctuatio...
1       [(심폐소생술, Noun), (교육, Noun), (일정, Noun), (안내, N...
2       [(EU, Alpha), (연구소, Noun), (인문사회연구사업단, Noun), ...
3       [(인도연구소, Noun), (HK, Alpha), (+, Punctuation),...
4       [(FBS, Alpha), (외대교육방송국, Noun), (78, Number), ...
                              ...                        
1668    [(안녕하세요, Adjective), (서울학생상담센터, Noun), (입니다, A...
1670    [(2021, Number), (학년도, Noun), (2, Number), (학기...
1671    [(안녕하세요, Adjective), (서울학생상담센터, Noun), (입니다, A...
1672    [(안녕하세요, Adjective), (서울, Noun), (학생, Noun), (...
1675    [(2021, Number), (학년도, Noun), (2, Number), (학기...
Name: tagged_content, Length: 1327, dtype: object


<ipython-input-13-ebe3e9c5a046>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gongji1_copy['tagged_content'] = gongji1_copy['cleaned_content'].apply(tokenize_sentences)


In [ ]:
'''gongji1_copy.loc[47,'tagged_content']'''

"gongji1_copy.loc[47,'tagged_content']"

In [ ]:
gongji1_copy.loc[35,'cleaned_content']

'법조인 양성을 위한HUFS Pre-Law School실원 모집2025학년도 법학전문대학원로스쿨진학을 희망하는 외대생을 지원하는Hufs Pre-Law School에서 실원을 선발하기 위한 시험을 다음과 같이 시행하오니 관심있는 학생들의 많은 참여바랍니다-다 음-1시 험 명 Hufs Pre-Law School입실 시험2시험일시2024 3 24일 0900~14003시험장소인문과학관1301호 강의실4응시자격본교 재학생수료생 또는 졸업생5시험과목법학적성시험LEET및 면접6신청접수 2024 3 7목 ~ 3 19회7신청방법입실신청서 및자기소개서작성 후 이메일접수 8시험일정구 분시 간비 고시험 장소입실 완료0850신분증필기구인문과학관1301호언어이해0900101030문70분추리논증1045125040문125분9기타추후 일정 개별통보실원 지원내용-기본법학 특강논술 특강 LEET모의고사 지원 면접지도 등-문의  '

In [ ]:
okt = Okt()
words = " ".join(gongji1_copy['cleaned_content'].tolist())
words = okt.morphs(words,stem=True)


In [ ]:
#55만개정도
len(words)

557346

In [ ]:
# 형태소 분석기 초기화
okt = Okt()

# 텍스트에서 명사만 추출
nouns = "".join(gongji1_copy['cleaned_content'].tolist())
nouns = okt.nouns(nouns)

# 명사 추출 완료 후 결과를 변수에 저장


In [ ]:
len(nouns)

325556

In [ ]:
nouns

['글로벌',
 '명상',
 '집단',
 '상담',
 '금모닝',
 '마음',
 '프로그램',
 '안내',
 '학생',
 '상담',
 '센터',
 '대해',
 '보고',
 '체험',
 '수',
 '금모닝',
 '마음',
 '집단',
 '상담',
 '명상',
 '마음',
 '관심',
 '학생',
 '분',
 '참여',
 '식목일',
 '실시',
 '후',
 '만족도',
 '조사',
 '참여',
 '모든',
 '분',
 '다육',
 '화분',
 '선물',
 '일',
 '시',
 '금',
 '택',
 '상',
 '명상',
 '대해',
 '관심',
 '글로벌',
 '캠퍼스',
 '재학생',
 '라면',
 '누구나',
 '회차별',
 '선착순',
 '명장',
 '소',
 '추후',
 '공지',
 '예정',
 '대면',
 '참여',
 '방법',
 '신청',
 '코드',
 '선착순',
 '마감',
 '문',
 '의',
 '학생',
 '상담',
 '센터',
 '학년도',
 '학기',
 '명상',
 '집단',
 '상담',
 '금모닝',
 '마음',
 '긴장',
 '때',
 '스트레스',
 '때',
 '주의',
 '집중',
 '때',
 '활용',
 '수',
 '명상',
 '전문가',
 '직접',
 '체험',
 '시간',
 '일시',
 '금',
 '택',
 '대상',
 '명상',
 '대해',
 '관심',
 '글로벌',
 '캠퍼스',
 '재학생',
 '라면',
 '누구나',
 '회차별',
 '선착순',
 '명장',
 '추후',
 '공지',
 '예정',
 '대면',
 '참여',
 '방법',
 '신청',
 '코드',
 '선착순',
 '마감',
 '참여',
 '후',
 '만족도',
 '조사',
 '참여',
 '분',
 '식목일',
 '힐링',
 '다육',
 '화분',
 '선물',
 '문의',
 '학생',
 '상담',
 '센터',
 '심폐소생술',
 '교육',
 '일정',
 '안내',
 '서울캠퍼스',
 '보건실',
 '교내',
 '구성원',
 '응급처치',
 '능력',
 '향상',
 '

In [ ]:
words

['[',
 '글로벌',
 ']',
 '2024-1',
 '명상',
 '집단',
 '상담',
 '＇',
 '금모닝',
 '마음',
 '챙기다',
 '＇',
 '프로그램',
 '안내',
 '학생',
 '상담',
 '센터',
 '에서는',
 '다가오다',
 '4월',
 '5일',
 '명상에',
 '대해',
 '배우다',
 '보고',
 '체험',
 '해보다',
 '수',
 '있다',
 '<',
 '금모닝',
 '마음',
 '챙기다',
 '>',
 '집단',
 '상담',
 '이',
 '열리다',
 '명상',
 '과',
 '마음',
 '챙기다',
 '관심',
 '있다',
 '학생',
 '분',
 '들',
 '의',
 '많다',
 '참여',
 '부탁드리다',
 '~',
 '식목일',
 '을',
 '맞다',
 '실시',
 '후',
 '만족도',
 '조사',
 '에',
 '참여',
 '해주다',
 '모든',
 '분',
 '들',
 '께',
 '"',
 '다육',
 '이',
 '화분',
 '"',
 '을',
 '선물',
 '로',
 '드리다',
 '~',
 '일',
 '시',
 '20240405',
 '금',
 '1000~1050',
 'A',
 'time',
 '또는',
 '1100~1150',
 'B',
 'time',
 '택',
 '1',
 '대다',
 '상',
 '명상',
 '에',
 '대해',
 '관심',
 '있다',
 '글로벌',
 '캠퍼스',
 '재학생',
 '이',
 '라면',
 '누구나',
 '회차별',
 '선착순',
 '10',
 '명장',
 '소',
 '추후',
 '공지',
 '예정',
 '대면',
 '참여',
 '방법',
 'HUFSAbility',
 '신청',
 'QR',
 '코드',
 '선착순',
 '마감',
 '문',
 '의',
 '학생',
 '상담',
 '센터',
 '2024',
 '학년도',
 '1',
 '학기',
 '명상',
 '집단',
 '상담',
 '금모닝',
 '마음',
 '챙기다',
 '긴장',
 '되다',
 '불안하다',
 '때',
 

In [ ]:
#395614약 39만개
remove_one_word = [x for x in words if len(x)>1 and len(x) < 15]
len(remove_one_word)

395614

In [ ]:
remove_nouns =  [x for x in nouns if len(x)>1 and len(x) < 15]
len(remove_nouns)

279789

In [ ]:
from collections import Counter
frequent = Counter(remove_one_word).most_common()
frequent

[('하다', 9509),
 ('되다', 3085),
 ('신청', 3076),
 ('교육', 2794),
 ('있다', 2777),
 ('학생', 2706),
 ('교수', 2661),
 ('으로', 2528),
 ('지원', 2446),
 ('제출', 2181),
 ('프로그램', 2149),
 ('에서', 2066),
 ('대학', 1995),
 ('학기', 1969),
 ('참여', 1598),
 ('모집', 1476),
 ('선발', 1473),
 ('경우', 1467),
 ('기간', 1413),
 ('진행', 1392),
 ('방법', 1372),
 ('학과', 1365),
 ('한국외국어대학교', 1343),
 ('활동', 1338),
 ('안내', 1314),
 ('센터', 1288),
 ('접수', 1249),
 ('가능', 1246),
 ('예정', 1206),
 ('바라다', 1185),
 ('서류', 1163),
 ('2023', 1162),
 ('확인', 1143),
 ('상담', 1135),
 ('또는', 1082),
 ('대상', 1079),
 ('내용', 1063),
 ('일정', 1046),
 ('관련', 1036),
 ('발표', 1032),
 ('한국외대', 986),
 ('이다', 985),
 ('문의', 927),
 ('학부', 926),
 ('납부', 921),
 ('대학원', 910),
 ('글로벌', 909),
 ('온라인', 897),
 ('홈페이지', 892),
 ('시간', 888),
 ('전공', 885),
 ('사항', 873),
 ('면접', 869),
 ('까지', 855),
 ('사업', 848),
 ('2022', 843),
 ('of', 827),
 ('등록', 808),
 ('인도', 807),
 ('학년도', 802),
 ('수업', 797),
 ('한국', 785),
 ('연구', 783),
 ('이상', 782),
 ('하고', 773),
 ('작성', 766),
 ('2023년', 765)

In [ ]:
from collections import Counter
frequent = Counter(remove_nouns).most_common()
frequent

[('신청', 3073),
 ('교육', 2788),
 ('학생', 2703),
 ('교수', 2659),
 ('지원', 2439),
 ('제출', 2181),
 ('프로그램', 2144),
 ('대학', 1995),
 ('학기', 1969),
 ('참여', 1598),
 ('선발', 1473),
 ('모집', 1471),
 ('경우', 1467),
 ('기간', 1412),
 ('진행', 1390),
 ('방법', 1371),
 ('학과', 1365),
 ('활동', 1336),
 ('한국외국어대학교', 1318),
 ('안내', 1314),
 ('센터', 1287),
 ('가능', 1246),
 ('접수', 1245),
 ('예정', 1200),
 ('서류', 1163),
 ('확인', 1143),
 ('상담', 1134),
 ('대상', 1073),
 ('내용', 1057),
 ('일정', 1045),
 ('관련', 1036),
 ('발표', 1031),
 ('한국외대', 983),
 ('학부', 926),
 ('문의', 925),
 ('납부', 920),
 ('글로벌', 907),
 ('대학원', 903),
 ('홈페이지', 892),
 ('온라인', 888),
 ('시간', 887),
 ('전공', 883),
 ('사항', 873),
 ('면접', 869),
 ('사업', 848),
 ('등록', 808),
 ('인도', 803),
 ('학년도', 802),
 ('수업', 797),
 ('한국', 785),
 ('연구', 782),
 ('이상', 773),
 ('작성', 766),
 ('재학생', 753),
 ('문화', 749),
 ('과정', 710),
 ('아래', 706),
 ('국제', 692),
 ('운영', 669),
 ('서울캠퍼스', 644),
 ('시험', 638),
 ('영어', 634),
 ('기준', 630),
 ('해당', 629),
 ('등록금', 623),
 ('학교', 616),
 ('반드시', 612),
 ('주제', 

In [ ]:
with open('./stopwords_gongji.txt', 'r') as f:
    list_file = f.readlines()
stopwords = list_file[0].split(",")
remove_stopwords = [x for x in remove_one_word if x not in stopwords]
#45만개 -> 35만개->33만개
len(remove_stopwords)

339661

In [ ]:
with open('./stopwords_gongji.txt', 'r') as f:
    list_file = f.readlines()
stopwords = list_file[0].split(",")
remove_stopwords_nouns = [x for x in remove_nouns if x not in stopwords]
#45만개 -> 35만개->33만개
len(remove_stopwords_nouns)

266989

In [ ]:
#한번만 등장하는 오타인애들도 제거
# remove_stopwords 리스트에서 각 단어의 등장 빈도를 세기
word_counts = Counter(remove_stopwords)

# 빈도가 1회 이상인 단어들만 추출하여 새로운 리스트에 추가
filtered_words = [word for word in remove_stopwords if word_counts[word] > 1]
words = filtered_words
#33만2천개(7천개 정도 줄었다)
len(filtered_words)

332007

In [ ]:
#한번만 등장하는 오타인애들도 제거
# remove_stopwords 리스트에서 각 단어의 등장 빈도를 세기
nouns_counts = Counter(remove_stopwords_nouns)

# 빈도가 1회 이상인 단어들만 추출하여 새로운 리스트에 추가
filtered_nouns = [nouns for nouns in remove_stopwords_nouns if nouns_counts[nouns] > 1]
nouns = filtered_nouns
#26만개
len(filtered_nouns)

264272

In [ ]:
#불용어 처리 후 빈도확인
Counter(filtered_words).most_common()

[('신청', 3076),
 ('교육', 2794),
 ('학생', 2706),
 ('교수', 2661),
 ('지원', 2446),
 ('제출', 2181),
 ('프로그램', 2149),
 ('대학', 1995),
 ('학기', 1969),
 ('참여', 1598),
 ('모집', 1476),
 ('선발', 1473),
 ('경우', 1467),
 ('기간', 1413),
 ('진행', 1392),
 ('방법', 1372),
 ('학과', 1365),
 ('한국외국어대학교', 1343),
 ('활동', 1338),
 ('안내', 1314),
 ('접수', 1249),
 ('가능', 1246),
 ('예정', 1206),
 ('서류', 1163),
 ('2023', 1162),
 ('확인', 1143),
 ('상담', 1135),
 ('대상', 1079),
 ('내용', 1063),
 ('일정', 1046),
 ('관련', 1036),
 ('발표', 1032),
 ('한국외대', 986),
 ('문의', 927),
 ('학부', 926),
 ('납부', 921),
 ('대학원', 910),
 ('글로벌', 909),
 ('온라인', 897),
 ('홈페이지', 892),
 ('전공', 885),
 ('사항', 873),
 ('면접', 869),
 ('사업', 848),
 ('2022', 843),
 ('등록', 808),
 ('인도', 807),
 ('학년도', 802),
 ('수업', 797),
 ('한국', 785),
 ('연구', 783),
 ('이상', 782),
 ('작성', 766),
 ('2023년', 765),
 ('재학생', 753),
 ('문화', 749),
 ('과정', 710),
 ('아래', 708),
 ('국제', 692),
 ('the', 690),
 ('따르다', 679),
 ('운영', 669),
 ('많다', 661),
 ('2022년', 646),
 ('서울캠퍼스', 645),
 ('시험', 639),
 ('영어', 634)

In [ ]:
Counter(nouns).most_common()

[('신청', 3073),
 ('교육', 2788),
 ('학생', 2703),
 ('교수', 2659),
 ('지원', 2439),
 ('제출', 2181),
 ('프로그램', 2144),
 ('대학', 1995),
 ('학기', 1969),
 ('참여', 1598),
 ('선발', 1473),
 ('모집', 1471),
 ('경우', 1467),
 ('기간', 1412),
 ('진행', 1390),
 ('방법', 1371),
 ('학과', 1365),
 ('활동', 1336),
 ('한국외국어대학교', 1318),
 ('안내', 1314),
 ('가능', 1246),
 ('접수', 1245),
 ('예정', 1200),
 ('서류', 1163),
 ('확인', 1143),
 ('상담', 1134),
 ('대상', 1073),
 ('내용', 1057),
 ('일정', 1045),
 ('관련', 1036),
 ('발표', 1031),
 ('한국외대', 983),
 ('학부', 926),
 ('문의', 925),
 ('납부', 920),
 ('글로벌', 907),
 ('대학원', 903),
 ('홈페이지', 892),
 ('온라인', 888),
 ('전공', 883),
 ('사항', 873),
 ('면접', 869),
 ('사업', 848),
 ('등록', 808),
 ('인도', 803),
 ('학년도', 802),
 ('수업', 797),
 ('한국', 785),
 ('연구', 782),
 ('이상', 773),
 ('작성', 766),
 ('재학생', 753),
 ('문화', 749),
 ('과정', 710),
 ('아래', 706),
 ('국제', 692),
 ('운영', 669),
 ('서울캠퍼스', 644),
 ('시험', 638),
 ('영어', 634),
 ('기준', 630),
 ('해당', 629),
 ('등록금', 623),
 ('학교', 616),
 ('주제', 606),
 ('서울', 603),
 ('불가', 593),
 ('관심', 59

In [ ]:
# remove_stopwords 리스트를 Counter로 변환하여 가장 적게 나오는 순서대로 출력
least_common_words = Counter(filtered_words).most_common()[::-1]

# 결과 출력
print(least_common_words)

[('20211005', 2), ('码并点击相应活动链接联系方式', 2), ('扫描', 2), ('点击本海报点击申请书链接', 2), ('日星期四申请方法', 2), ('日星期五', 2), ('点申请期间', 2), ('5-7', 2), ('下午', 2), ('日星期四', 2), ('克服学业困难的具体学习方法', 2), ('学业适应', 2), ('调整方法', 2), ('心理适应', 2), ('更好的适应本校的生活', 2), ('권종', 2), ('액면가', 2), ('현찰', 2), ('환전소', 2), ('봉투', 2), ('10/24일', 2), ('10/01', 2), ('·>', 2), ('방위산업', 2), ('카본', 2), ('북극권', 2), ('류인태', 2), ('역사연구', 2), ('김성묵', 2), ('김형주', 2), ('윤리학', 2), ('11/5', 2), ('기탁금', 2), ('선거운동', 2), ('HUFSBUS', 2), ('승객', 2), ('App', 2), ('20211018월', 2), ('Unity', 2), ('Talk', 2), ('adjust', 2), ('help', 2), ('conduct', 2), ('선포', 2), ('공약', 2), ('이노베이션', 2), ('SK', 2), ('operation', 2), ('buses', 2), ('Garak', 2), ('Seongnam', 2), ('Street', 2), ('Seokgwan', 2), ('timetable', 2), ('Details', 2), ('dongSeoul', 2), ('Sunae', 2), ('across', 2), ('Airport', 2), ('Range', 2), ('Dormitory', 2), ('StationJam', 2), ('Bokjeong', 2), ('stationGlobal', 2), ('highwaySongpa', 2), ('Pangyo', 2), ('Guri', 2), ('workTaereung', 2), ('campu

In [ ]:
filtered_words

['글로벌',
 '2024-1',
 '명상',
 '집단',
 '상담',
 '금모닝',
 '마음',
 '챙기다',
 '프로그램',
 '안내',
 '학생',
 '상담',
 '에서는',
 '다가오다',
 '4월',
 '명상에',
 '배우다',
 '체험',
 '금모닝',
 '마음',
 '챙기다',
 '집단',
 '상담',
 '열리다',
 '명상',
 '마음',
 '챙기다',
 '관심',
 '학생',
 '많다',
 '참여',
 '부탁드리다',
 '식목일',
 '맞다',
 '실시',
 '만족도',
 '조사',
 '참여',
 '다육',
 '화분',
 '선물',
 '드리다',
 '20240405',
 'time',
 '1100~1150',
 'time',
 '명상',
 '관심',
 '글로벌',
 '캠퍼스',
 '재학생',
 '라면',
 '회차별',
 '선착순',
 '10',
 '명장',
 '공지',
 '예정',
 '대면',
 '참여',
 '방법',
 'HUFSAbility',
 '신청',
 '코드',
 '선착순',
 '마감',
 '학생',
 '상담',
 '2024',
 '학년도',
 '학기',
 '명상',
 '집단',
 '상담',
 '금모닝',
 '마음',
 '챙기다',
 '긴장',
 '불안하다',
 '스트레스',
 '심하다',
 '집중',
 '필요하다',
 '활용',
 '명상',
 '전문가',
 '배우다',
 '직접',
 '체험',
 '일시',
 '20240405',
 'time',
 '20240405',
 '1100~1150',
 'time',
 '대상',
 '명상',
 '관심',
 '글로벌',
 '캠퍼스',
 '재학생',
 '라면',
 '회차별',
 '선착순',
 '10',
 '명장',
 '공지',
 '예정',
 '대면',
 '참여',
 '방법',
 'HUFSAbility',
 '신청',
 '코드',
 '선착순',
 '마감',
 '참여',
 '만족도',
 '조사',
 '참여',
 '식목일',
 '맞다',
 '힐링',
 '다육',
 '화분',
 '선물',
 '드리다',


In [ ]:
len(nouns)

264272

In [ ]:
# 마지막 최종에 해도될듯 :외국어없애기
'''
!pip install googletrans==3.1.0a0
# 한국어, 영어 문자만 남기기
from googletrans import Translator
def clean_text(text):
    translator = Translator()
    detected = translator.detect(text)
    lang = detected.lang
    if lang in ['ko', 'en']:
        return text
    else:
        return ''
'''

### TF-IDF 및 텍스트랭크 참고링크
https://www.dinolabs.ai/m/288 텍스트랭크 적용하기

In [ ]:
from google.colab import files

# 파일을 쓰기 모드로 열고 데이터를 쓴 후 파일을 다운로드합니다.
with open('gongji1_words_list.txt', 'w', encoding='utf-8') as file:
    for words in filtered_words:
        line = ''.join(words)
        file.write(line + '\n')

# 다운로드할 파일을 지정합니다.
files.download('gongji1_words_list.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files

# 파일을 쓰기 모드로 열고 데이터를 쓴 후 파일을 다운로드합니다.
with open('gongji1_nouns_list.txt', 'w', encoding='utf-8') as file:
    for noun in nouns:
        line = ''.join(noun)
        file.write(line + '\n')

# 다운로드할 파일을 지정합니다.
files.download('gongji1_nouns_list.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#세션다운대비 이등분
from google.colab import files

# 리스트를 이등분하여 두 개의 부분으로 나눕니다.
split_index = len(filtered_words) // 2
part1 = filtered_words[:split_index]
part2 = filtered_words[split_index:]

# 첫 번째 부분을 파일로 저장합니다.
with open('gongji1_words_list_part1.txt', 'w', encoding='utf-8') as file:
    for words in part1:
        line = ''.join(words)
        file.write(line + '\n')

# 두 번째 부분을 파일로 저장합니다.
with open('gongji1_words_list_part2.txt', 'w', encoding='utf-8') as file:
    for words in part2:
        line = ''.join(words)
        file.write(line + '\n')

# 파일을 다운로드합니다.
files.download('gongji1_words_list_part1.txt')
files.download('gongji1_words_list_part2.txt')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# 파일 열기
with open('gongji1_words_list.txt', 'r', encoding='utf-8') as file:
    # 파일의 내용을 처음 5줄만 읽어와서 출력
    for _ in range(5):
        line = file.readline()
        print(line)

글로벌

2024-1

명상

집단

상담



In [ ]:
#명사 파일열기
with open('gongji1_nouns_list.txt','r',encoding = 'utf-8') as file:
  for _ in range(5):
    line = file.readline()
    print(line)

글로벌

명상

집단

상담

금모닝



In [ ]:
#tf-idf 넣어줄 사이킷런다운
!pip install -U scikit-learn
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer


In [ ]:
ts

NameError: name 'ts' is not defined

In [ ]:
#공지 읽어오기
with open("gongji1_words_list_part1.txt",'r',encoding = 'utf-8') as file:
  lines = file.readlines()
  data = [line.strip() for line in lines]
documents = data
documents[1]
vectorizer = CountVectorizer()

In [ ]:
#명사읽어오기
with open ("gongji1_nouns_list.txt", 'r', encoding = 'utf-8') as file:
  lines = file.readlines()
  data = [line.strip() for line in lines]
doc_nouns = data
doc_nouns[1]
vectorizer = CountVectorizer()

In [ ]:
# Document Term Matrix
dtm = vectorizer.fit_transform(documents)

# Term Freqeuncy
tf = pd.DataFrame(dtm.toarray(), columns=vectorizer.get_feature_names_out())
# Document Frequency
df = tf.astype(bool).sum(axis = 0)
# 문서 개수
D = len(tf)

NameError: name 'documents' is not defined

In [ ]:
# Inverse Document Frequency
idf = np.log((D) / (df+1))
# TF-IDF _연산안될시 두가지 : 불용어 처리 더추가하기 , 데이터 반 자르기
tfidf = tf * idf
tfidf = tfidf / np.linalg.norm(tfidf, axis = 1, keepdims = True)
tfidf

,000,0000,00289,004,0050,00분,01,0112,0126,01월,...,힌두,힌두교,힌디,힌디어,힐링,힐임,힘드다,힘쓰다,힘차다,歷史
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
165999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
166000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
166001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#nouns vectorize
dtm = vectorizer.fit_transform(doc_nouns)
tf = pd.DataFrame(dtm.toarray(), columns = vectorizer.get_feature_names_out())
df = tf.astype(bool).sum(axis = 0)
D = len(tf)

In [ ]:
# Inverse Document Frequency
idf = np.log((D) / (df+1))
# TF-IDF _연산안될시 두가지 : 불용어 처리 더추가하기 , 데이터 반 자르기
tfidf = tf * idf
tfidf = tfidf / np.linalg.norm(tfidf, axis = 1, keepdims = True)
tfidf

,가가,가감,가격,가계,가관,가구,가기,가까이,가나,가나다,...,히데,히로시마,히말라야,힌두,힌두교,힌두이즘,힌디,힌디어,힐링,힐임
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264267,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
264268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
264269,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
264270,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


AttributeError: 'TfidfVectorizer' object has no attribute 'dtypes'

### tfidf 2번째방법
https://www.dinolabs.ai/m/288

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import numpy as np
from scipy.sparse import csr_matrix
from konlpy.tag import Okt

tfidf = TfidfVectorizer()
cnt_vec = CountVectorizer()
graph_sentence = []



In [7]:
#명사읽어오기
with open ("gongji1_nouns_list.txt", 'r', encoding = 'utf-8') as file:
  lines = file.readlines()
  data = [line.strip() for line in lines]
doc_nouns = data
doc_nouns

['글로벌',
 '명상',
 '집단',
 '상담',
 '금모닝',
 '마음',
 '프로그램',
 '안내',
 '학생',
 '상담',
 '체험',
 '금모닝',
 '마음',
 '집단',
 '상담',
 '명상',
 '마음',
 '관심',
 '학생',
 '참여',
 '식목일',
 '실시',
 '만족도',
 '조사',
 '참여',
 '다육',
 '화분',
 '선물',
 '명상',
 '관심',
 '글로벌',
 '캠퍼스',
 '재학생',
 '라면',
 '회차별',
 '선착순',
 '명장',
 '공지',
 '예정',
 '대면',
 '참여',
 '방법',
 '신청',
 '코드',
 '선착순',
 '마감',
 '학생',
 '상담',
 '학년도',
 '학기',
 '명상',
 '집단',
 '상담',
 '금모닝',
 '마음',
 '긴장',
 '스트레스',
 '집중',
 '활용',
 '명상',
 '전문가',
 '직접',
 '체험',
 '일시',
 '대상',
 '명상',
 '관심',
 '글로벌',
 '캠퍼스',
 '재학생',
 '라면',
 '회차별',
 '선착순',
 '명장',
 '공지',
 '예정',
 '대면',
 '참여',
 '방법',
 '신청',
 '코드',
 '선착순',
 '마감',
 '참여',
 '만족도',
 '조사',
 '참여',
 '식목일',
 '힐링',
 '다육',
 '화분',
 '선물',
 '문의',
 '학생',
 '상담',
 '심폐소생술',
 '교육',
 '일정',
 '안내',
 '서울캠퍼스',
 '보건실',
 '교내',
 '구성원',
 '응급처치',
 '능력',
 '향상',
 '심폐소생술',
 '교육',
 '실시',
 '오니',
 '관심',
 '참여',
 '교육',
 '시작',
 '이후',
 '입실',
 '제한',
 '이수증',
 '발급',
 '불가',
 '심폐소생술',
 '교육',
 '일정',
 '교육',
 '신청',
 '안내',
 '교육',
 '일정',
 '교육',
 '장소교',
 '회관',
 '강연',
 '교육',
 '대상',
 '재학생',
 '선착순',
 '

In [8]:
import numpy as np
from scipy.sparse import csr_matrix

In [9]:
tfidf = TfidfVectorizer()
cnt_vec = CountVectorizer()
graph_sentence = []

In [10]:
def build_sent_graph(sentence):
    tfidf_mat = tfidf.fit_transform(sentence).toarray()

    return tfidf_mat

sent_graph = build_sent_graph(doc_nouns)

In [11]:
sent_graph

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [13]:
from scipy.sparse import csr_matrix

# tfidf_mat를 csr_matrix로 변환
tfidf_mat_sparse = csr_matrix(sent_graph)

# 전치행렬 계산
tfidf_mat_transpose = tfidf_mat_sparse.T

# 전치행렬 곱 계산
graph_sentence_sparse = tfidf_mat_sparse.dot(tfidf_mat_transpose)

In [14]:
# 희소 행렬의 일부를 샘플링하여 출력
print(graph_sentence_sparse[100:105, 100:105].toarray())

[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]


In [15]:
graph_sentence_sparse.shape

(264272, 264272)

In [16]:
#words graph
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer

def build_words_graph(sentence):
    cnt_vec_mat = normalize(cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
    vocab = cnt_vec.vocabulary_
    return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}

words_graph, idx2word = build_words_graph(doc_nouns)

In [17]:
words_graph

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [18]:
idx2word

{735: '글로벌',
 1771: '명상',
 5463: '집단',
 2686: '상담',
 749: '금모닝',
 1653: '마음',
 6247: '프로그램',
 3503: '안내',
 6350: '학생',
 5604: '체험',
 502: '관심',
 5504: '참여',
 3317: '식목일',
 3387: '실시',
 1678: '만족도',
 5143: '조사',
 1165: '다육',
 6641: '화분',
 2841: '선물',
 5802: '캠퍼스',
 4829: '재학생',
 1527: '라면',
 6724: '회차별',
 2856: '선착순',
 1783: '명장',
 446: '공지',
 3819: '예정',
 1254: '대면',
 2114: '방법',
 3362: '신청',
 5842: '코드',
 1634: '마감',
 6333: '학년도',
 6329: '학기',
 856: '긴장',
 3228: '스트레스',
 5467: '집중',
 6689: '활용',
 4898: '전문가',
 5423: '직접',
 4559: '일시',
 1263: '대상',
 6789: '힐링',
 1884: '문의',
 3424: '심폐소생술',
 539: '교육',
 4570: '일정',
 2804: '서울캠퍼스',
 2268: '보건실',
 522: '교내',
 590: '구성원',
 4224: '응급처치',
 1143: '능력',
 6512: '향상',
 3829: '오니',
 3297: '시작',
 4422: '이후',
 4632: '입실',
 5123: '제한',
 4333: '이수증',
 2083: '발급',
 2458: '불가',
 4769: '장소교',
 6703: '회관',
 122: '강연',
 1061: '내외',
 4978: '접수',
 775: '기간',
 4957: '전화',
 779: '기관',
 629: '국민',
 6567: '협회',
 118: '강사',
 4653: '자격증',
 3009: '소지',
 5256: '준비물

In [19]:
idx2word

{735: '글로벌',
 1771: '명상',
 5463: '집단',
 2686: '상담',
 749: '금모닝',
 1653: '마음',
 6247: '프로그램',
 3503: '안내',
 6350: '학생',
 5604: '체험',
 502: '관심',
 5504: '참여',
 3317: '식목일',
 3387: '실시',
 1678: '만족도',
 5143: '조사',
 1165: '다육',
 6641: '화분',
 2841: '선물',
 5802: '캠퍼스',
 4829: '재학생',
 1527: '라면',
 6724: '회차별',
 2856: '선착순',
 1783: '명장',
 446: '공지',
 3819: '예정',
 1254: '대면',
 2114: '방법',
 3362: '신청',
 5842: '코드',
 1634: '마감',
 6333: '학년도',
 6329: '학기',
 856: '긴장',
 3228: '스트레스',
 5467: '집중',
 6689: '활용',
 4898: '전문가',
 5423: '직접',
 4559: '일시',
 1263: '대상',
 6789: '힐링',
 1884: '문의',
 3424: '심폐소생술',
 539: '교육',
 4570: '일정',
 2804: '서울캠퍼스',
 2268: '보건실',
 522: '교내',
 590: '구성원',
 4224: '응급처치',
 1143: '능력',
 6512: '향상',
 3829: '오니',
 3297: '시작',
 4422: '이후',
 4632: '입실',
 5123: '제한',
 4333: '이수증',
 2083: '발급',
 2458: '불가',
 4769: '장소교',
 6703: '회관',
 122: '강연',
 1061: '내외',
 4978: '접수',
 775: '기간',
 4957: '전화',
 779: '기관',
 629: '국민',
 6567: '협회',
 118: '강사',
 4653: '자격증',
 3009: '소지',
 5256: '준비물

In [20]:
def get_ranks(graph, d=0.85): # d = damping factor
    A = graph
    matrix_size = A.shape[0]
    for id in range(matrix_size):
        A[id, id] = 0 # diagonal 부분을 0으로
        link_sum = np.sum(A[:,id]) # A[:, id] = A[:][id]
        if link_sum != 0:
            A[:, id] /= link_sum
        A[:, id] *= -d
        A[id, id] = 1

    B = (1-d) * np.ones((matrix_size, 1))
    ranks = np.linalg.solve(A, B) # 연립방정식 Ax = b
    return {idx: r[0] for idx, r in enumerate(ranks)}

In [ ]:
sent_rank_idx = get_ranks(graph_sentence_sparse)
sent_rank_idx

In [ ]:
sent_graph_shape = sent_graph.shape
print(sent_graph_shape)

(264272, 6791)


# 사용자 사전 고치기
- 아래의 절차 따르고 맨 아래 data write 쪽 정정해주기
- - 사용자 사전 만들기 (colab)
https://sirzzang.github.io/ai/AI-konlpy-userdic/

In [ ]:
#사용자 사전 찾아가기
os.chdir('/usr/local/lib/python3.10/dist-packages/konlpy/java')
os.getcwd()

'/usr/local/lib/python3.10/dist-packages/konlpy/java'

In [ ]:
os.makedirs('./temp_fix')
os.chdir('./temp_fix')
os.getcwd()

'/usr/local/lib/python3.10/dist-packages/konlpy/java/temp_fix'

In [ ]:
!jar xvf ../open-korean-text-2.1.0.jar

  created: META-INF/
 inflated: META-INF/MANIFEST.MF
  created: org/
  created: org/openkoreantext/
  created: org/openkoreantext/processor/
  created: org/openkoreantext/processor/normalizer/
  created: org/openkoreantext/processor/phrase_extractor/
  created: org/openkoreantext/processor/qa/
  created: org/openkoreantext/processor/stemmer/
  created: org/openkoreantext/processor/tokenizer/
  created: org/openkoreantext/processor/tools/
  created: org/openkoreantext/processor/util/
  created: org/openkoreantext/processor/util/adjective/
  created: org/openkoreantext/processor/util/adverb/
  created: org/openkoreantext/processor/util/auxiliary/
  created: org/openkoreantext/processor/util/freq/
  created: org/openkoreantext/processor/util/josa/
  created: org/openkoreantext/processor/util/noun/
  created: org/openkoreantext/processor/util/substantives/
  created: org/openkoreantext/processor/util/typos/
  created: org/openkoreantext/processor/util/verb/
 inflated: org/openkoreantext/pr

In [ ]:
pwd

'/usr/local/lib/python3.10/dist-packages/konlpy/java/temp_fix'

In [ ]:
# 사용자 사전 열기
with open(f"/usr/local/lib/python3.10/dist-packages/konlpy/java/temp_fix/org/openkoreantext/processor/util/noun/names.txt") as f:
    data = f.read()
data

'가몽\n가온\n갓세븐\n강새이\n게임닉가\n관우\n귀여미\n규\n김유이\n김준면\n까까런\n노컷\n누너예\n니노\n다마고치\n다이무스\n대학생\n데이브\n도요토미\n동운\n동이\n두주니\n디시인사이드\n디오\n라몹\n라스\n라옵\n멍구\n메이든\n명덕\n명량\n문민정부\n미네\n방엘리\n병헌\n붓다\n비정상회담\n빼빼로\n삼풍\n샤인온미\n성식\n성열\n세라문\n세라복\n세종대왕\n손권\n손책\n쇼미더머니\n쇼챔\n순규\n스라소니\n신동아\n신쓰패밀리\n신아라\n아베\n안상홍\n안홍준\n여누\n여랑\n여포\n연합\n오꾸닭\n요섭\n웃찾사\n원식\n유병언\n유비\n유이\n윤기형\n이나단\n이명박\n이완용\n임창용\n자괴\n자니윤\n자대련\n자유\n재중이\n전교조\n정윤회\n제갈량\n조자룡\n조조\n준면\n지오디\n지존파\n진영오\n차작가\n차트\n창섭\n챠트\n첸\n코르사주\n하무열\n하용파쿠\n혁재\n현이\n현태\n혜미\n'

In [ ]:
data += '한국외국어대학교\n한국외대\n안내\종합정보시스템\공고\n이수\n학기\n대하여\n아래\n수강편람\n이중전공\n수강신청함\n수강신청\n예비수강신청\n교무처장\n수강료\n등록기간\n수강신청자\n한하다\n미수강신청자\n납부불가\n핫링크\n고지서출력\n가상계좌\n붙임파일\n모바일앱\n내역\n인터넷뱅킹\n텔레뱅킹\n개시일\n신청\n마감\n마감일\n수강취소\n수강변경\종합정보화면\n계절학기\n부분환불\n계절학기\학년도\n접수일\n학석사\n학사\n석사\n연계\n과정생\n접수처\n학사종합지원센터\n양캠퍼스\n통번역\n이수\n미배정\n부전공\여석\n이수\n지원\n학사종합지원센터\n한국예술종합학교\n특성화\n수강편람\n실용외국어\n공동교양\n여석\졸업\n사정\n결과\n확정자\n불가자\n개강일\n우리대학교\n우리대학\n안내문\n학위\n학위증\n코로나바이러스\n신종코로나바이러스\nHUFS\n양캠퍼스\n서울캠퍼스\n글로벌캠퍼스\n서울글로벌캠퍼스\n서울·글로벌캠퍼스\n우리은행\n단과대학\n졸업시험\n논문\n제적\n졸업요건\n지역학사\n지역전문가과정\n수료졸업대기\nELLT학과\n영어학과\n아랍어과\n미디어커뮤니케이션학부\n영어전공\n영미문학문화학과\n태국어과\n경제학부\nEICC학과\n영어통번역학과\n베트남어과\n국제통상학과\이란어과\n경영학부\n프랑스어학부\n인도어과\n영어교육과\n독일어과\n터키아제르바이잔어과\n프랑스어교육과\n노어과\n몽골어과\n독일어교육과\n스페인어과\n중국언어문화학부\n한국어교육과\n이탈리아어과\n중국외교통상학부\n국제학부\n포르투갈어과\n그리스불가리아학과\n일본언어문화학부\n사회과학전공\n네덜란드어과\n융합일본지역학부\nLD전공\n스칸디나비아어과\n정치외교학과\nLT전공\n말레이인도네시아어과\n행정학과\n서양어대학\n아시아언어문화대학\n중국어대학\n일본어대학\n사회과학\n상경\n경영\n국제학부\n사회과학전공\n사범대학\n교직\n실용외국어\n군입대\n입영통지서\n미비\n미충족\n등록휴학\n휴학\n전역일\n증빙서류\n1학기\n2학기\n개강\n종강\n학사일정\n행사명\n조치사항\n참고사항\n공고\n부서명\n학년도\n개설학과\n학수번호\교과목명\시간\n교강사명\n강의시간\n폐강공고\n일정\n단과대\n배부처\n문의\n동유럽학대학\n국제지역대학\n어문학관\n통번역대학\n인문대학\n경상대학\n자연과학대학\n학생회관\n공과대학\n바이오메디컬공학부\n졸업증명서\n문의\n전화\문의전화\n원스톱서비스센터\n졸업예정증명서\n편입생\n신입생\n신편입생\n아래\nQR코드\n지양하다\n투명성\n공정성\n외국인교환학생\n외국인방문학생\n첫학기\n영어진단평가\n원격\n비대면\ne-Class\neclass\n이클래스\n안내하다\n전자출결\n전자출결시스템\n체계적인\n엄정성\n공고\n코로나19\n일반휴학\n신청\n별도문의\nCOVID-19\nCovid19\covid19\n중간시험\기말시험\n응시인원\nCommunicativeEnglish\nLS\nL/S\n우리대학\n콜로키움\nEU연구소\n인문사회연구사업단\n온·오프라인\n온오프라인\nHK+사업단\n국제관\n인도연구소\n패널티\n수습국원\n추가모집\n게시물\n외대교육방송국\n교수학습개발원\nFBS\주한브라질대사관\n세계여성의날\스마트도서관\n휠라아쿠쉬네트홀\n영화\n상영\n한국어자막\대학혁신지원사업\n아이디어톤\n수도권역\n미래대학투모로우솔루션\n미네르바헬스장\n교육생\n휴관\n금모닝\n글로벌캠퍼스\n누구나\n회차별\n보건실\n대한적십자사\n헌혈증\n헌혈자\n성격유형검사\n핵심역량통합관리시스템\nHufsability\n교과\n비교과\n서울학생상담센터\n추후공지\n청년몽땅정보통\n마이페이지\n문의처\n다산콜센터\특별전형\n일반전형\n면접전형\n등록예치금\n예치금\n모집\n학과\n전공\nTESOL대학원\n세미오시스연구센터\n서울학생상담센터\n전문\n경영대학원\n해당일\n접수마감일\n해당학기\n졸업자\n졸업예정자\n외국대학\n소지자\n한국어능력시험\n입증서류\n출신학교\n주한공관영사\n입증서류\n학사학위\n학사\n최종학력\n증명\n학위증서\n개인정보이용및제공동의서\n기타서류\n해당자\n추천서\n경력증명서\n외국국적지원자\n성적증명서\n총점\n사이버외대\n군인공제회직원\n학교교직원\n첫학기\n입학금\n운영위원회\n전형료\n우편접수\n입학원서\n본인날인\n서명함\n수험표\n지참\n제출서류\n외국어교육센터\nK-Zone\nNoshow\n출석차감\n1교시\n2교시\n3교시\n4교시\n5교시\n6교시\n7교시\n8교시\n9교시\n10교시\n11교시\n12교시\n작문\n대학영어\n중간시험\n보충수업\n추가모집\n창업자\nAI/SW\n모집공고\n모집분야\n일반창업\n스펙업\n졸업대기자\n수료생\nSW중심대학사업단\nAI교육원\nSW관련학과\n컴퓨터공학부\n정보통신공학부\nGBT전공\nELLT학과\nAI융합전공\n데이터사이언스전공\n바이오데이터사이언스전공\nICT&AI세부모듈\n수혜제외\n초과학기생\n초과학기\n9학기\n1학기\n2학기\n3학기\n4학기\n5학기\n6학기\n7학기\n8학기\n지원사항\n팀별\n개인별\n사업자등록증\n최종보고서\n학기말\n학기초\n전세계\n삼성그룹\nLG그룹\n주기적\n인증테스트\n비교과\n굿즈\n코로나19\n경제적\n지원\n타지역\n이에\n랜선나눔캠퍼스\n활동개요\n신청기간\n비합격\n비선발자\n최종합격자\n활동불가자\n활동혜택\n자기소개서\n1부\n학습계획서\n증빙자료\n재학증명서\n1차합격\n2차합격\n스캔\n활동비\소득세\n지방소득세\n문의처\n추후공지\n공공인재개발원\n개인사물함\n행정입법고시\n2차모의고사\n수험서적\n사민재\n세미나실\n국제학생회\n본교생\n외국인교환학생\n정기총회\n뵨교재학생\n버디프로그램\n홍보팀\n 기획컬처팀\n국제교류처\n국제교류처장\n불합격자\n코로나-19\n양성판정\n음성판정\n양성\n음성\n최종합격자\nQ&A\n필참\n퇴출\n정기활동\n이메일접수\n시험일정\n구분\n시간\n비고\n시험장소\n입실완료\n인문과학관\n졸업유예생\n학부생들\n코로나-19\n글로벌캠퍼스\n보건실\n학생부종합전형\n합격수기\n선배들\n글로벌창업지원단\n창업인재양성센터\n창업동아리\n미디어커뮤니케이션대학원\n대학원장\nCulture&Technology융합대학\n지원자격\n지원서\n접수\n평가요소\n영자신문사\nTheArgus\n수습기자\n디아거스\n공식홈페이지\n공식인스타그램\n사회봉사센터\n국제학사\n우편접수\n서울시동대문구이문로107\n대학안전관리계획\n붙임팡리\n안전보건관리팀\n동원번역상\n'
# 파일 새롭게 저장
with open("/usr/local/lib/python3.10/dist-packages/konlpy/java/temp_fix/org/openkoreantext/processor/util/noun/names.txt", 'w') as f:
    f.write(data)
data

'가몽\n가온\n갓세븐\n강새이\n게임닉가\n관우\n귀여미\n규\n김유이\n김준면\n까까런\n노컷\n누너예\n니노\n다마고치\n다이무스\n대학생\n데이브\n도요토미\n동운\n동이\n두주니\n디시인사이드\n디오\n라몹\n라스\n라옵\n멍구\n메이든\n명덕\n명량\n문민정부\n미네\n방엘리\n병헌\n붓다\n비정상회담\n빼빼로\n삼풍\n샤인온미\n성식\n성열\n세라문\n세라복\n세종대왕\n손권\n손책\n쇼미더머니\n쇼챔\n순규\n스라소니\n신동아\n신쓰패밀리\n신아라\n아베\n안상홍\n안홍준\n여누\n여랑\n여포\n연합\n오꾸닭\n요섭\n웃찾사\n원식\n유병언\n유비\n유이\n윤기형\n이나단\n이명박\n이완용\n임창용\n자괴\n자니윤\n자대련\n자유\n재중이\n전교조\n정윤회\n제갈량\n조자룡\n조조\n준면\n지오디\n지존파\n진영오\n차작가\n차트\n창섭\n챠트\n첸\n코르사주\n하무열\n하용파쿠\n혁재\n현이\n현태\n혜미\n한국외국어대학교\n한국외대\n안내\\종합정보시스템\\공고\n이수\n학기\n대하여\n아래\n수강편람\n이중전공\n수강신청함\n수강신청\n예비수강신청\n교무처장\n수강료\n등록기간\n수강신청자\n한하다\n미수강신청자\n납부불가\n핫링크\n고지서출력\n가상계좌\n붙임파일\n모바일앱\n내역\n인터넷뱅킹\n텔레뱅킹\n개시일\n신청\n마감\n마감일\n수강취소\n수강변경\\종합정보화면\n계절학기\n부분환불\n계절학기\\학년도\n접수일\n학석사\n학사\n석사\n연계\n과정생\n접수처\n학사종합지원센터\n양캠퍼스\n통번역\n이수\n미배정\n부전공\\여석\n이수\n지원\n학사종합지원센터\n한국예술종합학교\n특성화\n수강편람\n실용외국어\n공동교양\n여석\\졸업\n사정\n결과\n확정자\n불가자\n개강일\n우리대학교\n우리대학\n안내문\n학위\n학위증\n코로나바이러스\n신종코로나바이러스\nHUFS\n양캠퍼스\n서울캠퍼스\n글로벌캠퍼스\n서울글로벌캠퍼스\n서울·글로벌캠퍼스\n우리은행\n단과대학\n졸업시험\n논문\n제적\n졸업요건\n지역학사\n지역전문가과정\n수료졸

In [ ]:
data

'가몽\n가온\n갓세븐\n강새이\n게임닉가\n관우\n귀여미\n규\n김유이\n김준면\n까까런\n노컷\n누너예\n니노\n다마고치\n다이무스\n대학생\n데이브\n도요토미\n동운\n동이\n두주니\n디시인사이드\n디오\n라몹\n라스\n라옵\n멍구\n메이든\n명덕\n명량\n문민정부\n미네\n방엘리\n병헌\n붓다\n비정상회담\n빼빼로\n삼풍\n샤인온미\n성식\n성열\n세라문\n세라복\n세종대왕\n손권\n손책\n쇼미더머니\n쇼챔\n순규\n스라소니\n신동아\n신쓰패밀리\n신아라\n아베\n안상홍\n안홍준\n여누\n여랑\n여포\n연합\n오꾸닭\n요섭\n웃찾사\n원식\n유병언\n유비\n유이\n윤기형\n이나단\n이명박\n이완용\n임창용\n자괴\n자니윤\n자대련\n자유\n재중이\n전교조\n정윤회\n제갈량\n조자룡\n조조\n준면\n지오디\n지존파\n진영오\n차작가\n차트\n창섭\n챠트\n첸\n코르사주\n하무열\n하용파쿠\n혁재\n현이\n현태\n혜미\n한국외국어대학교\n한국외대\n안내\\종합정보시스템\\공고\n이수\n학기\n대하여\n아래\n수강편람\n이중전공\n수강신청함\n수강신청\n예비수강신청\n교무처장\n수강료\n등록기간\n수강신청자\n한하다\n미수강신청자\n납부불가\n핫링크\n고지서출력\n가상계좌\n붙임파일\n모바일앱\n내역\n인터넷뱅킹\n텔레뱅킹\n개시일\n신청\n마감\n마감일\n수강취소\n수강변경\\종합정보화면\n계절학기\n부분환불\n계절학기\\학년도\n접수일\n학석사\n학사\n석사\n연계\n과정생\n접수처\n학사종합지원센터\n양캠퍼스\n통번역\n이수\n미배정\n부전공\\여석\n이수\n지원\n학사종합지원센터\n한국예술종합학교\n특성화\n수강편람\n실용외국어\n공동교양\n여석\\졸업\n사정\n결과\n확정자\n불가자\n개강일\n우리대학교\n우리대학\n안내문\n학위\n학위증\n코로나바이러스\n신종코로나바이러스\nHUFS\n양캠퍼스\n서울캠퍼스\n글로벌캠퍼스\n서울글로벌캠퍼스\n서울·글로벌캠퍼스\n우리은행\n단과대학\n졸업시험\n논문\n제적\n졸업요건\n지역학사\n지역전문가과정\n수료졸

In [ ]:
!jar cvf ../open-korean-text-2.1.0.jar *

added manifest
ignoring entry META-INF/
ignoring entry META-INF/MANIFEST.MF
adding: META-INF/maven/(in = 0) (out= 0)(stored 0%)
adding: META-INF/maven/org.openkoreantext/(in = 0) (out= 0)(stored 0%)
adding: META-INF/maven/org.openkoreantext/open-korean-text/(in = 0) (out= 0)(stored 0%)
adding: META-INF/maven/org.openkoreantext/open-korean-text/pom.properties(in = 119) (out= 110)(deflated 7%)
adding: META-INF/maven/org.openkoreantext/open-korean-text/pom.xml(in = 9127) (out= 2208)(deflated 75%)
adding: org/(in = 0) (out= 0)(stored 0%)
adding: org/openkoreantext/(in = 0) (out= 0)(stored 0%)
adding: org/openkoreantext/processor/(in = 0) (out= 0)(stored 0%)
adding: org/openkoreantext/processor/KoreanPosJava.class(in = 2457) (out= 1241)(deflated 49%)
adding: org/openkoreantext/processor/tools/(in = 0) (out= 0)(stored 0%)
adding: org/openkoreantext/processor/tools/UpdateAllTheExamples$.class(in = 4383) (out= 1794)(deflated 59%)
adding: org/openkoreantext/processor/tools/CreateParsingExamples

In [ ]:
import os
from konlpy.tag import Okt
new_dir = '/content'

# 작업 디렉토리 변경
os.chdir(new_dir)
okt = Okt()
print(okt.nouns('송이레만세')) # ['송이레만세']
print(okt.morphs('김재경자경')) # ['김재경자경']
print(okt.morphs("한국외국어대학교"))

['송이', '레만', '세']
['김재경', '자', '경']
['한국외국어대학교']


In [ ]:
pwd

# ---------------------------------------------------------- 사전 수정하는 법 --------------------------------------------------

# ------------------------- 수연님 공지 병합하기 ----------------------
- 하단 계속

In [ ]:
import pandas as pd

In [ ]:
gongji1_copy

In [ ]:
len(gongji1_copy)

In [ ]:
gongji_2.loc[150]

In [ ]:
gongji_3 =  pd.read_csv('./drive/MyDrive/csv모음/gongji_fix/noti_gongji3.csv')
len(gongji_3)

In [ ]:
gongji_1 = pd.read_csv('./drive/MyDrive/csv모음/gongji_fix/merge_file_final(병합전).csv')
len(gongji_1)

In [ ]:
# 세 개의 DataFrame을 하나로 병합
merged_df = pd.concat([gongji_1, gongji_2, gongji_3], ignore_index=True)

# 병합된 DataFrame을 새로운 인코딩으로 저장
merged_df.to_csv('merged_gongji_test.csv', encoding='utf-8-sig', index=False)

len(merged_df)

In [ ]:
test = pd.read_csv("/content/merged_gongji_test.csv")

In [ ]:
test.loc[5000]

# 번역기 사용하여 외국어처리하기


In [ ]:
!pip install googletrans==3.1.0a0

# 한국어, 영어 문자만 남기기
from googletrans import Translator

def clean_text(text):
    translator = Translator()
    detected = translator.detect(text)
    lang = detected.lang
    if lang in ['ko', 'en']:
        return text
    else:
        return ''

In [ ]:
gongji1_copy['content'] = gongji1_copy['content'].apply(clean_text)